In [64]:
import os
import json
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from utils import check_env_variable, check_file_exists, generate_tp_name

from gptravel.core.travel_planner.openai_engine import ChatGPTravelEngine
from gptravel.core.travel_planner.prompt import PlainTravelPrompt
from gptravel.core.travel_planner.travel_engine import TravelPlanJSON
from gptravel.core.services.geocoder import GeoCoder

from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.runnables.passthrough import RunnableAssign

from gptravel.core.utils.regex_tool import JsonExtractor
from utils import from_json_to_text
from typing import Dict, Any
from operator import itemgetter

load_dotenv()

check_env_variable('OPENAI_API_KEY')
examples_folder = os.path.join('examples')

OPENAI_API_KEY loaded correctly


In [2]:
PARAMETERS = {
    'departure_place': 'Milan',
    'departure_date': datetime(2024, 8, 30),
    'destination_place': 'Berlin',
    'end_date': datetime(2024, 9, 6),
    'gpt_params': {
        'model': 'gpt-4o-mini',
        'max_tokens': 600,
        'temperature': 0,
    },
}

PARAMETERS['n_travel_days'] = (PARAMETERS['end_date'] - PARAMETERS['departure_date']).days
PARAMETERS['tp_filename'] = generate_tp_name(PARAMETERS) + '.json'

geocoder = GeoCoder()

PARAMETERS['destination_distance'] = geocoder.location_distance(location_name_1=PARAMETERS['departure_place'],
                                                  location_name_2=PARAMETERS['destination_place'])

print('Travel parameters', PARAMETERS)

[2024-09-01 11:58:34,411 - DEBUG] Querying coordinates for milan
[2024-09-01 11:58:34,411 - DEBUG] Downloading new Location for milan: Start
[2024-09-01 11:58:34,679 - DEBUG] Downloading new Location for milan: Complete
[2024-09-01 11:58:34,679 - DEBUG] Querying coordinates for berlin
[2024-09-01 11:58:34,684 - DEBUG] Downloading new Location for berlin: Start
[2024-09-01 11:58:34,949 - DEBUG] Downloading new Location for berlin: Complete


Travel parameters {'departure_place': 'Milan', 'departure_date': datetime.datetime(2024, 8, 30, 0, 0), 'destination_place': 'Berlin', 'end_date': datetime.datetime(2024, 9, 6, 0, 0), 'gpt_params': {'model': 'gpt-4o-mini', 'max_tokens': 600, 'temperature': 0}, 'n_travel_days': 7, 'tp_filename': 'tp_dep_Milan_arr_Berlin_n_d7.json', 'destination_distance': 841.6720665233777}


In [3]:
gpt_model = ChatGPTravelEngine(
    **PARAMETERS['gpt_params']
)
prompt = PlainTravelPrompt(**PARAMETERS)
print('Prompt', prompt.prompt)

Prompt Generate a JSON with inside a travel plan of 7 days for a person who wants to visit Berlin from
Milan. The structure of the JSON must be the following:
{"Day x": {"City": ["activity 1", "activity2",...], "City": ["activity 1", ...]} , "Day x+1": {"City": [...], "City": [...]} } 
where City field is the city visited in the corresponding Day.


### travel generation with old framework

plain_generation = 'Here’s a JSON representation of a 5-day travel plan for a person visiting Berlin from Milan:\n\n```json\n{\n  "Day 1": {\n    "Berlin": [\n      "Arrive in Berlin from Milan",\n      "Check into hotel",\n      "Visit the Brandenburg Gate",\n      "Stroll through Tiergarten",\n      "Dinner at a local restaurant"\n    ]\n  },\n  "Day 2": {\n    "Berlin": [\n      "Visit the Berlin Wall Memorial",\n      "Explore the East Side Gallery",\n      "Lunch at a nearby café",\n      "Visit Museum Island",\n      "Evening at the Berlin Cathedral"\n    ]\n  },\n  "Day 3": {\n    "Berlin": [\n      "Tour the Reichstag Building",\n      "Visit the Holocaust Memorial",\n      "Lunch in the Mitte district",\n      "Explore the shopping area at Kurfürstendamm",\n      "Dinner in a traditional German beer garden"\n    ]\n  },\n  "Day 4": {\n    "Berlin": [\n      "Day trip to Potsdam to visit Sanssouci Palace",\n      "Explore the gardens of Sanssouci",\n      "Lunch in Potsdam",\n      "Visit the Dutch Quarter",\n      "Return to Berlin for the evening"\n    ]\n  },\n  "Day 5": {\n    "Berlin": [\n      "Visit the Pergamon Museum",\n      "Explore the vibrant neighborhood of Kreuzberg",\n      "Lunch at a street food market",\n      "Visit the Checkpoint Charlie Museum",\n      "Depart Berlin for Milan"\n    ]\n  }\n}\n```\n\nThis travel plan provides a structured itinerary for a visitor to Berlin, detailing activities for each day.'

In [4]:
file_path = os.path.join(examples_folder, PARAMETERS['tp_filename'])
if check_file_exists(file_path):
    print('Loading travel plan inside:',file_path)
    with open(file_path, 'r') as json_data:
        tp_json = json_data.read()
        json_data.close()
    travel_plan = TravelPlanJSON(
        **PARAMETERS,
        n_days=PARAMETERS['n_travel_days'],
        travel_plan_json=json.loads(tp_json),
        json_keys_depth_map={'city': 1, 'day': 0},
    )
else:
    print('Generating the new travel plan')
    #travel_plan = gpt_model.get_travel_plan_json(prompt=prompt)
    with open(file_path, "w") as jfile:
        json.dump(travel_plan.travel_plan, jfile)
    print('Saved travel plan in', file_path)

print('\nGenerated travel plan', travel_plan.travel_plan)

Loading travel plan inside: examples\tp_dep_Milan_arr_Berlin_n_d5.json

Generated travel plan {'Day 1': {'Berlin': ['Arrive in Berlin from Milan', 'Check into the hotel', 'Visit the Brandenburg Gate', 'Explore the Reichstag Building', 'Dinner at a local German restaurant']}, 'Day 2': {'Berlin': ['Visit the Berlin Wall Memorial', 'Explore the East Side Gallery', 'Lunch at a nearby cafÃ©', 'Visit Museum Island', 'Enjoy a river cruise on the Spree']}, 'Day 3': {'Berlin': ['Visit the Pergamon Museum', 'Explore the Berlin Cathedral', 'Lunch in the Nikolaiviertel district', 'Visit the Jewish Museum', 'Dinner in Kreuzberg']}, 'Day 4': {'Berlin': ['Day trip to Potsdam', 'Visit Sanssouci Palace', 'Explore the gardens of Sanssouci', 'Lunch in Potsdam', 'Return to Berlin and enjoy a night out in Friedrichshain']}, 'Day 5': {'Berlin': ['Visit the Topography of Terror', 'Explore Checkpoint Charlie', 'Lunch at a local market', 'Last-minute shopping at KurfÃ¼rstendamm', 'Depart from Berlin to Milan']

## Test with LangChain

In [4]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign

from gptravel.core.utils.regex_tool import JsonExtractor
from utils import from_json_to_text

prompt_generation = """Generate a JSON with inside a travel plan of {n_travel_days} days for a person who wants to visit {destination_place} from
          {departure_place}. The structure of the JSON must be the following:
          {{"Day x": {{"City x": ["activity 1 (time slot proxy)", "activity2 (time slot proxy)",...], "City y": ["activity 1 (time slot proxy)", ...]}} , "Day x+1": {{"City x": [...], "City y": [...]}} }} 
          where City field is the city visited in the corresponding Day."""

travel_plan_prompt_generator = ChatPromptTemplate.from_messages(
    [SystemMessage(content=("You are an helpful travel plan assistant that generates travel plans based on user's requests"
                          "It is important that the assistant respects the requested output formats."
                          "You must only answer with the requested travel plan, avoiding any kind of conversational sentence.")),
    HumanMessagePromptTemplate.from_template(prompt_generation),
    ]
)

openai_agent = ChatOpenAI(**PARAMETERS['gpt_params'])

travel_generator_chain_agent = travel_plan_prompt_generator | openai_agent | StrOutputParser() | RunnableLambda(from_json_to_text)

tp_output_json = travel_generator_chain_agent.invoke(PARAMETERS)

#tp_output = '{\n  "Day 1": {\n    "Berlin": [\n      "Arrive in Berlin from Milan",\n      "Check into hotel",\n      "Visit Brandenburg Gate",\n      "Explore the Reichstag Building",\n      "Dinner at a local restaurant"\n    ]\n  },\n  "Day 2": {\n    "Berlin": [\n      "Visit the Berlin Wall Memorial",\n      "Explore the East Side Gallery",\n      "Lunch at a nearby café",\n      "Visit Museum Island",\n      "Evening stroll in Alexanderplatz"\n    ]\n  },\n  "Day 3": {\n    "Berlin": [\n      "Visit the Holocaust Memorial",\n      "Explore the Topography of Terror museum",\n      "Lunch in Kreuzberg",\n      "Visit the Berlin Cathedral",\n      "Enjoy nightlife in Friedrichshain"\n    ]\n  },\n  "Day 4": {\n    "Berlin": [\n      "Day trip to Potsdam",\n      "Visit Sanssouci Palace",\n      "Explore the gardens of Sanssouci",\n      "Lunch in Potsdam",\n      "Return to Berlin for dinner"\n    ]\n  },\n  "Day 5": {\n    "Berlin": [\n      "Visit the Berlin Zoo",\n      "Explore the Kurfürstendamm shopping street",\n      "Lunch at a café",\n      "Visit the Charlottenburg Palace",\n      "Depart from Berlin to Milan"\n    ]\n  }\n}'

print(tp_output_json)

{'Day 1': {'Berlin': ['Arrive in Berlin (morning)', 'Check into hotel (midday)', 'Visit Brandenburg Gate (afternoon)', 'Dinner at a local restaurant (evening)']}, 'Day 2': {'Berlin': ['Visit Berlin Wall Memorial (morning)', 'Explore Museum Island (midday)', 'Lunch at a café (afternoon)', 'Visit Pergamon Museum (afternoon)', 'Evening stroll along the Spree River (evening)']}, 'Day 3': {'Berlin': ['Visit Reichstag Building (morning)', 'Explore Tiergarten Park (midday)', 'Lunch in the park (afternoon)', 'Visit Victory Column (afternoon)', 'Dinner in Kreuzberg (evening)']}, 'Day 4': {'Berlin': ['Visit East Side Gallery (morning)', 'Explore Friedrichshain district (midday)', 'Lunch at a local eatery (afternoon)', 'Visit Jewish Museum (afternoon)', 'Attend a local event or concert (evening)']}, 'Day 5': {'Berlin': ['Visit Charlottenburg Palace (morning)', 'Explore the palace gardens (midday)', 'Lunch at a nearby café (afternoon)', 'Visit the Berlin Zoo (afternoon)', 'Dinner in the Wilmersdor

In [5]:
def from_text_to_json_travel_plan(text: str) -> Dict[Any, Any]:
    regex = JsonExtractor()
    dict_string = regex(text)[0]
    try:
        json_object = json.loads(dict_string)
    except json.decoder.JSONDecodeError:
        json_object = json.loads(
            r"{}".format(dict_string[0].replace("'", '"'))
        )
    return json_object


def build_travel_plan_object(_dict):
    return TravelPlanJSON(departure_place=_dict['departure_place'],
                          destination_place=_dict['destination_place'],
                          n_days=_dict['n_travel_days'],
                          travel_plan_json=_dict['travel_plan_json_parsed'],
                          json_keys_depth_map={'city': 1, 'day': 0})


PARAMETERS = {
    'departure_place': 'Milan',
    'departure_date': datetime(2024, 6, 10),
    'destination_place': 'Berlin',
    'end_date': datetime(2024, 6, 15),
    'gpt_params': {
        'model': 'gpt-4o-mini',
        'max_tokens': 400,
        'temperature': 0.5,
    },
}

PARAMETERS['n_travel_days'] = (PARAMETERS['end_date'] - PARAMETERS['departure_date']).days
PARAMETERS['tp_filename'] = generate_tp_name(PARAMETERS) + '.json'


prompt_generation = """Generate a JSON with inside a travel plan of {n_travel_days} days for a person who wants to visit {destination_place} from
          {departure_place}. The structure of the JSON must be the following:
          {{"Day x": {{"City x": ["activity 1", "activity2",...], "City y": ["activity 1", ...]}} , "Day x+1": {{"City x": [...], "City y": [...]}} }} 
          where City field is the city visited in the corresponding Day."""

travel_plan_prompt_generator = ChatPromptTemplate.from_messages(
    [SystemMessage(content=("You are an helpful travel plan assistant that generates travel plans based on user's requests"
                          "It is important that the assistant respects the requested output formats."
                          "You must only answer with the requested travel plan, avoiding any kind of conversational sentence.")),
    HumanMessagePromptTemplate.from_template(prompt_generation),
    ]
)

openai_agent = ChatOpenAI(**PARAMETERS['gpt_params'])

travel_generator_chain_agent = travel_plan_prompt_generator | openai_agent | StrOutputParser() | RunnableLambda(from_text_to_json_travel_plan)

travel_generator_chain_agent = (
    RunnableMap(
        {
            'departure_place': itemgetter('departure_place'),
            'destination_place': itemgetter('destination_place'),
            'n_travel_days': itemgetter('n_travel_days'),
            'travel_plan_json_parsed': travel_generator_chain_agent
        }
        
    ) |
    RunnableLambda(build_travel_plan_object)  # Convert JSON to TravelPlanJSON object
)

#generated_travel_plan = travel_generator_chain_agent.invoke(PARAMETERS)

In [5]:
FLIGHT_API_PARAMETERS = {
    'departure_id': 'LIN',
    'arrival_id': 'BER',
    'gl': 'it', # country of the engine search
    'hl': 'en', # language
    'currency': 'eur',
    'type': '1', # 1 = round trip, 2 = one way, 3 = multicity
    'outbound_date': PARAMETERS['departure_date'].strftime('%Y-%m-%d'),
    'return_date': PARAMETERS['end_date'].strftime('%Y-%m-%d'),
    'travel_class': 1, # 1 = economy, 2 = premium economy, 3 = business, 4 = first
    'adults': 1, # number of adults
    'children': 0, # number of children
    'stops': 0, # number of scali
    'bags': 0, # number of carry bags
}

print(FLIGHT_API_PARAMETERS)

{'departure_id': 'LIN', 'arrival_id': 'BER', 'gl': 'it', 'hl': 'en', 'currency': 'eur', 'type': '1', 'outbound_date': '2024-08-30', 'return_date': '2024-09-06', 'travel_class': 1, 'adults': 1, 'children': 0, 'stops': 0, 'bags': 0}


## FLIGHTS

In [6]:
import json

file_path = os.path.join(examples_folder, 'tp_dep_Milan_arr_Berlin_n_d5_flights.json')

#api_costs = json.loads(file_path)
DEPARTURE_FLIGHTS_INFO = {
  "search_metadata": {
    "id": "66d0d9e34a4a1da0cb59d1df",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/1042647e3d2ce6a2/66d0d9e34a4a1da0cb59d1df.json",
    "created_at": "2024-08-29 20:28:19 UTC",
    "processed_at": "2024-08-29 20:28:19 UTC",
    "google_flights_url": "https://www.google.com/travel/flights?hl=en&gl=us&curr=EUR&tfs=CBwQAhogEgoyMDI0LTA4LTMwKAFqBwgBEgNMSU5yBwgBEgNCRVJCAQFIAXABmAEC",
    "raw_html_file": "https://serpapi.com/searches/1042647e3d2ce6a2/66d0d9e34a4a1da0cb59d1df.html",
    "prettify_html_file": "https://serpapi.com/searches/1042647e3d2ce6a2/66d0d9e34a4a1da0cb59d1df.prettify",
    "total_time_taken": 2.46
  },
  "search_parameters": {
    "engine": "google_flights",
    "hl": "en",
    "gl": "us",
    "type": "2",
    "departure_id": "LIN",
    "arrival_id": "BER",
    "outbound_date": "2024-08-30",
    "travel_class": 1,
    "adults": 1,
    "stops": 1,
    "currency": "EUR"
  },
  "other_flights": [
    {
      "flights": [
        {
          "departure_airport": {
            "name": "Milan Linate Airport",
            "id": "LIN",
            "time": "2024-08-30 06:55"
          },
          "arrival_airport": {
            "name": "Berlin Brandenburg Airport",
            "id": "BER",
            "time": "2024-08-30 08:35"
          },
          "duration": 100,
          "airplane": "Airbus A320",
          "airline": "easyJet",
          "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
          "travel_class": "Economy",
          "flight_number": "U2 5068",
          "legroom": "29 in",
          "extensions": [
            "Below average legroom (29 in)",
            "Carbon emissions estimate: 84 kg"
          ]
        }
      ],
      "total_duration": 100,
      "carbon_emissions": {
        "this_flight": 85000,
        "typical_for_this_route": 90000,
        "difference_percent": -6
      },
      "price": 121,
      "type": "One way",
      "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
      "booking_token": "WyJDalJJWWpabFZWWlZjMmxhWTNOQlFrUm5VWGRDUnkwdExTMHRMUzB0YjNsamFYTXlNa0ZCUVVGQlIySlJNbVZWUWxOekxVRkJFZ1pWTWpVd05qZ2FDZ2lSWGhBQ0dnTkZWVkk0SEhDaGFBPT0iLFtbIkxJTiIsIjIwMjQtMDgtMzAiLCJCRVIiLG51bGwsIlUyIiwiNTA2OCJdXV0="
    },
    {
      "flights": [
        {
          "departure_airport": {
            "name": "Milan Linate Airport",
            "id": "LIN",
            "time": "2024-08-30 11:00"
          },
          "arrival_airport": {
            "name": "Berlin Brandenburg Airport",
            "id": "BER",
            "time": "2024-08-30 12:40"
          },
          "duration": 100,
          "airplane": "Airbus A319",
          "airline": "easyJet",
          "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
          "travel_class": "Economy",
          "flight_number": "U2 5064",
          "legroom": "29 in",
          "extensions": [
            "Below average legroom (29 in)",
            "Carbon emissions estimate: 94 kg"
          ],
          "often_delayed_by_over_30_min": True
        }
      ],
      "total_duration": 100,
      "carbon_emissions": {
        "this_flight": 95000,
        "typical_for_this_route": 90000,
        "difference_percent": 6
      },
      "price": 136,
      "type": "One way",
      "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
      "booking_token": "WyJDalJJWWpabFZWWlZjMmxhWTNOQlFrUm5VWGRDUnkwdExTMHRMUzB0YjNsamFYTXlNa0ZCUVVGQlIySlJNbVZWUWxOekxVRkJFZ1pWTWpVd05qUWFDZ2lmYWhBQ0dnTkZWVkk0SEhEV2RRPT0iLFtbIkxJTiIsIjIwMjQtMDgtMzAiLCJCRVIiLG51bGwsIlUyIiwiNTA2NCJdXV0="
    },
    {
      "flights": [
        {
          "departure_airport": {
            "name": "Milan Linate Airport",
            "id": "LIN",
            "time": "2024-08-30 16:55"
          },
          "arrival_airport": {
            "name": "Berlin Brandenburg Airport",
            "id": "BER",
            "time": "2024-08-30 18:35"
          },
          "duration": 100,
          "airplane": "Airbus A319",
          "airline": "easyJet",
          "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
          "travel_class": "Economy",
          "flight_number": "U2 5072",
          "legroom": "29 in",
          "extensions": [
            "Below average legroom (29 in)",
            "Carbon emissions estimate: 94 kg"
          ],
          "often_delayed_by_over_30_min": True
        }
      ],
      "total_duration": 100,
      "carbon_emissions": {
        "this_flight": 95000,
        "typical_for_this_route": 90000,
        "difference_percent": 6
      },
      "price": 176,
      "type": "One way",
      "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
      "booking_token": "WyJDalJJWWpabFZWWlZjMmxhWTNOQlFrUm5VWGRDUnkwdExTMHRMUzB0YjNsamFYTXlNa0ZCUVVGQlIySlJNbVZWUWxOekxVRkJFZ1pWTWpVd056SWFDd2lOaVFFUUFob0RSVlZTT0J4dzdaY0IiLFtbIkxJTiIsIjIwMjQtMDgtMzAiLCJCRVIiLG51bGwsIlUyIiwiNTA3MiJdXV0="
    }
  ],
  "price_insights": {
    "lowest_price": 121,
    "price_level": "high",
    "typical_price_range": [
      45,
      110
    ],
    "price_history": [
      [
        1719698400,
        30
      ],
      [
        1719784800,
        30
      ],
      [
        1719871200,
        31
      ],
      [
        1719957600,
        31
      ],
      [
        1720044000,
        31
      ],
      [
        1720130400,
        31
      ],
      [
        1720216800,
        32
      ],
      [
        1720303200,
        32
      ],
      [
        1720389600,
        32
      ],
      [
        1720476000,
        34
      ],
      [
        1720562400,
        31
      ],
      [
        1720648800,
        31
      ],
      [
        1720735200,
        31
      ],
      [
        1720821600,
        31
      ],
      [
        1720908000,
        31
      ],
      [
        1720994400,
        31
      ],
      [
        1721080800,
        31
      ],
      [
        1721167200,
        31
      ],
      [
        1721253600,
        32
      ],
      [
        1721340000,
        32
      ],
      [
        1721426400,
        32
      ],
      [
        1721512800,
        34
      ],
      [
        1721599200,
        34
      ],
      [
        1721685600,
        36
      ],
      [
        1721772000,
        36
      ],
      [
        1721858400,
        37
      ],
      [
        1721944800,
        37
      ],
      [
        1722031200,
        37
      ],
      [
        1722117600,
        40
      ],
      [
        1722204000,
        40
      ],
      [
        1722290400,
        40
      ],
      [
        1722376800,
        46
      ],
      [
        1722463200,
        46
      ],
      [
        1722549600,
        47
      ],
      [
        1722636000,
        47
      ],
      [
        1722722400,
        47
      ],
      [
        1722808800,
        47
      ],
      [
        1722895200,
        54
      ],
      [
        1722981600,
        56
      ],
      [
        1723068000,
        56
      ],
      [
        1723154400,
        60
      ],
      [
        1723240800,
        60
      ],
      [
        1723327200,
        60
      ],
      [
        1723413600,
        60
      ],
      [
        1723500000,
        60
      ],
      [
        1723586400,
        60
      ],
      [
        1723672800,
        62
      ],
      [
        1723759200,
        62
      ],
      [
        1723845600,
        62
      ],
      [
        1723932000,
        67
      ],
      [
        1724018400,
        67
      ],
      [
        1724104800,
        67
      ],
      [
        1724191200,
        70
      ],
      [
        1724277600,
        72
      ],
      [
        1724364000,
        78
      ],
      [
        1724450400,
        78
      ],
      [
        1724536800,
        80
      ],
      [
        1724623200,
        80
      ],
      [
        1724709600,
        82
      ],
      [
        1724796000,
        84
      ],
      [
        1724882400,
        121
      ]
    ]
  }
}

RETURN_FLIGHTS_INFO = {
  "search_metadata": {
    "id": "66d0d93d2a4882b04476a8a5",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/1042647e3d2ce6a2/66d0d93d2a4882b04476a8a5.json",
    "created_at": "2024-08-29 20:25:33 UTC",
    "processed_at": "2024-08-29 20:25:33 UTC",
    "google_flights_url": "https://www.google.com/travel/flights?hl=en&gl=us&curr=EUR&tfs=CBwQAhogEgoyMDI0LTA5LTA1KAFqBwgBEgNCRVJyBwgBEgNMSU5CAQFIAXABmAEC",
    "raw_html_file": "https://serpapi.com/searches/1042647e3d2ce6a2/66d0d93d2a4882b04476a8a5.html",
    "prettify_html_file": "https://serpapi.com/searches/1042647e3d2ce6a2/66d0d93d2a4882b04476a8a5.prettify",
    "total_time_taken": 1.18
  },
  "search_parameters": {
    "engine": "google_flights",
    "hl": "en",
    "gl": "us",
    "type": "2",
    "departure_id": "BER",
    "arrival_id": "LIN",
    "outbound_date": "2024-09-05",
    "travel_class": 1,
    "adults": 1,
    "stops": 1,
    "currency": "EUR"
  },
  "other_flights": [
    {
      "flights": [
        {
          "departure_airport": {
            "name": "Berlin Brandenburg Airport",
            "id": "BER",
            "time": "2024-09-05 08:20"
          },
          "arrival_airport": {
            "name": "Milan Linate Airport",
            "id": "LIN",
            "time": "2024-09-05 10:00"
          },
          "duration": 100,
          "airplane": "Airbus A320",
          "airline": "easyJet",
          "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
          "travel_class": "Economy",
          "flight_number": "U2 5063",
          "legroom": "29 in",
          "extensions": [
            "Below average legroom (29 in)",
            "Carbon emissions estimate: 84 kg"
          ],
          "often_delayed_by_over_30_min": True
        }
      ],
      "total_duration": 100,
      "carbon_emissions": {
        "this_flight": 85000,
        "typical_for_this_route": 90000,
        "difference_percent": -6
      },
      "price": 74,
      "type": "One way",
      "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
      "booking_token": "WyJDalJJV1doS2RuWkNiMU5aYUhOQlFrNXRPRUZDUnkwdExTMHRMUzB0YjNsamFXd3hOVUZCUVVGQlIySlJNbFEwUjNGTk1uZEJFZ1pWTWpVd05qTWFDZ2lHT1JBQ0dnTkZWVkk0SEhDWVB3PT0iLFtbIkJFUiIsIjIwMjQtMDktMDUiLCJMSU4iLG51bGwsIlUyIiwiNTA2MyJdXV0="
    },
    {
      "flights": [
        {
          "departure_airport": {
            "name": "Berlin Brandenburg Airport",
            "id": "BER",
            "time": "2024-09-05 14:25"
          },
          "arrival_airport": {
            "name": "Milan Linate Airport",
            "id": "LIN",
            "time": "2024-09-05 16:10"
          },
          "duration": 105,
          "airplane": "Airbus A320",
          "airline": "easyJet",
          "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
          "travel_class": "Economy",
          "flight_number": "U2 5071",
          "legroom": "29 in",
          "extensions": [
            "Below average legroom (29 in)",
            "Carbon emissions estimate: 87 kg"
          ]
        }
      ],
      "total_duration": 105,
      "carbon_emissions": {
        "this_flight": 88000,
        "typical_for_this_route": 90000,
        "difference_percent": -2
      },
      "price": 91,
      "type": "One way",
      "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
      "booking_token": "WyJDalJJV1doS2RuWkNiMU5aYUhOQlFrNXRPRUZDUnkwdExTMHRMUzB0YjNsamFXd3hOVUZCUVVGQlIySlJNbFEwUjNGTk1uZEJFZ1pWTWpVd056RWFDZ2lxUmhBQ0dnTkZWVkk0SEhEeVRRPT0iLFtbIkJFUiIsIjIwMjQtMDktMDUiLCJMSU4iLG51bGwsIlUyIiwiNTA3MSJdXV0="
    },
    {
      "flights": [
        {
          "departure_airport": {
            "name": "Berlin Brandenburg Airport",
            "id": "BER",
            "time": "2024-09-05 21:10"
          },
          "arrival_airport": {
            "name": "Milan Linate Airport",
            "id": "LIN",
            "time": "2024-09-05 22:50"
          },
          "duration": 100,
          "airplane": "Airbus A320",
          "airline": "easyJet",
          "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
          "travel_class": "Economy",
          "flight_number": "U2 5073",
          "legroom": "29 in",
          "extensions": [
            "Below average legroom (29 in)",
            "Carbon emissions estimate: 84 kg"
          ],
          "often_delayed_by_over_30_min": True
        }
      ],
      "total_duration": 100,
      "carbon_emissions": {
        "this_flight": 85000,
        "typical_for_this_route": 90000,
        "difference_percent": -6
      },
      "price": 101,
      "type": "One way",
      "airline_logo": "https://www.gstatic.com/flights/airline_logos/70px/U2.png",
      "booking_token": "WyJDalJJV1doS2RuWkNiMU5aYUhOQlFrNXRPRUZDUnkwdExTMHRMUzB0YjNsamFXd3hOVUZCUVVGQlIySlJNbFEwUjNGTk1uZEJFZ1pWTWpVd056TWFDZ2lTVGhBQ0dnTkZWVkk0SEhER1ZnPT0iLFtbIkJFUiIsIjIwMjQtMDktMDUiLCJMSU4iLG51bGwsIlUyIiwiNTA3MyJdXV0="
    }
  ],
  "price_insights": {
    "lowest_price": 74,
    "price_level": "typical",
    "typical_price_range": [
      45,
      150
    ],
    "price_history": [
      [
        1719698400,
        35
      ],
      [
        1719784800,
        35
      ],
      [
        1719871200,
        43
      ],
      [
        1719957600,
        43
      ],
      [
        1720044000,
        43
      ],
      [
        1720130400,
        37
      ],
      [
        1720216800,
        37
      ],
      [
        1720303200,
        37
      ],
      [
        1720389600,
        37
      ],
      [
        1720476000,
        37
      ],
      [
        1720562400,
        37
      ],
      [
        1720648800,
        39
      ],
      [
        1720735200,
        39
      ],
      [
        1720821600,
        39
      ],
      [
        1720908000,
        39
      ],
      [
        1720994400,
        39
      ],
      [
        1721080800,
        43
      ],
      [
        1721167200,
        45
      ],
      [
        1721253600,
        50
      ],
      [
        1721340000,
        51
      ],
      [
        1721426400,
        51
      ],
      [
        1721512800,
        51
      ],
      [
        1721599200,
        51
      ],
      [
        1721685600,
        56
      ],
      [
        1721772000,
        56
      ],
      [
        1721858400,
        50
      ],
      [
        1721944800,
        50
      ],
      [
        1722031200,
        56
      ],
      [
        1722117600,
        56
      ],
      [
        1722204000,
        56
      ],
      [
        1722290400,
        56
      ],
      [
        1722376800,
        50
      ],
      [
        1722463200,
        51
      ],
      [
        1722549600,
        51
      ],
      [
        1722636000,
        56
      ],
      [
        1722722400,
        56
      ],
      [
        1722808800,
        56
      ],
      [
        1722895200,
        56
      ],
      [
        1722981600,
        50
      ],
      [
        1723068000,
        50
      ],
      [
        1723154400,
        50
      ],
      [
        1723240800,
        50
      ],
      [
        1723327200,
        50
      ],
      [
        1723413600,
        50
      ],
      [
        1723500000,
        59
      ],
      [
        1723586400,
        59
      ],
      [
        1723672800,
        59
      ],
      [
        1723759200,
        59
      ],
      [
        1723845600,
        52
      ],
      [
        1723932000,
        52
      ],
      [
        1724018400,
        52
      ],
      [
        1724104800,
        54
      ],
      [
        1724191200,
        60
      ],
      [
        1724277600,
        62
      ],
      [
        1724364000,
        56
      ],
      [
        1724450400,
        56
      ],
      [
        1724536800,
        56
      ],
      [
        1724623200,
        56
      ],
      [
        1724709600,
        60
      ],
      [
        1724796000,
        62
      ],
      [
        1724882400,
        74
      ]
    ]
  }
}

In [7]:
def get_best_flights(api_response, index_offset: int = 0):
    try:
        best_flights = api_response['best_flights']
    except KeyError:
        best_flights = api_response['other_flights']
    print('Number of best_flights', len(best_flights))
    return [{'flight_id': i + index_offset,
             'departure_time': x['flights'][0]['departure_airport']['time'], 
            'arrival_time': x['flights'][0]['arrival_airport']['time'],
             'departure_airport': x['flights'][0]['departure_airport']['name'],
             'arrival_airport': x['flights'][0]['arrival_airport']['name'],
            'airline': x['flights'][0]['airline'],
            'price': x['price'],
            'total_duration': x['total_duration']} for i, x in enumerate(best_flights)]

from copy import copy

In [8]:
best_departure_flights_info = get_best_flights(DEPARTURE_FLIGHTS_INFO)
best_return_flights_info = get_best_flights(RETURN_FLIGHTS_INFO, index_offset=len(best_departure_flights_info))

flights_info_json = copy(best_departure_flights_info)
flights_info_json += best_return_flights_info

df = pd.DataFrame(best_departure_flights_info)
df = pd.concat([df, pd.DataFrame(best_return_flights_info)]).drop('total_duration', axis=1).reset_index(drop=True)
df_markdown = df.to_markdown(index=False)
print(df_markdown)

Number of best_flights 3
Number of best_flights 3
|   flight_id | departure_time   | arrival_time     | departure_airport          | arrival_airport            | airline   |   price |
|------------:|:-----------------|:-----------------|:---------------------------|:---------------------------|:----------|--------:|
|           0 | 2024-08-30 06:55 | 2024-08-30 08:35 | Milan Linate Airport       | Berlin Brandenburg Airport | easyJet   |     121 |
|           1 | 2024-08-30 11:00 | 2024-08-30 12:40 | Milan Linate Airport       | Berlin Brandenburg Airport | easyJet   |     136 |
|           2 | 2024-08-30 16:55 | 2024-08-30 18:35 | Milan Linate Airport       | Berlin Brandenburg Airport | easyJet   |     176 |
|           3 | 2024-09-05 08:20 | 2024-09-05 10:00 | Berlin Brandenburg Airport | Milan Linate Airport       | easyJet   |      74 |
|           4 | 2024-09-05 14:25 | 2024-09-05 16:10 | Berlin Brandenburg Airport | Milan Linate Airport       | easyJet   |      91 |
|           

In [9]:
generated_travel_plan = "{'Day 1': {'Berlin': ['Arrive in Berlin from Milan', 'Check into hotel', 'Visit Brandenburg Gate', 'Explore Tiergarten Park', 'Dinner at a local restaurant']}, 'Day 2': {'Berlin': ['Visit the Berlin Wall Memorial', 'Explore Museum Island', 'Lunch at a café', 'Visit the Pergamon Museum', 'Evening stroll along the Spree River']}, 'Day 3': {'Berlin': ['Tour the Reichstag Building', 'Visit the Holocaust Memorial', 'Lunch in Potsdamer Platz', 'Explore the Topography of Terror museum', 'Dinner in Kreuzberg']}, 'Day 4': {'Berlin': ['Day trip to Potsdam', 'Visit Sanssouci Palace', 'Explore the gardens', 'Lunch in Potsdam', 'Return to Berlin for dinner']}, 'Day 5': {'Berlin': ['Visit East Side Gallery', 'Explore Friedrichshain district', 'Lunch at a local market', 'Shopping at Alexanderplatz', 'Depart Berlin for Milan']}}"

generated_travel_plan = tp_output_json#"{'Day 1': {'Berlin': ['Arrive in Berlin from Milan', 'Check into hotel (30 minutes)', 'Visit Brandenburg Gate (1 hour)', 'Explore the Reichstag Building (1.5 hours)', 'Dinner at a local restaurant (2 hours)']}, 'Day 2': {'Berlin': ['Visit the Berlin Wall Memorial (1.5 hours)', 'Explore Museum Island (3 hours)', 'Lunch at a nearby café (1 hour)', 'Visit the Pergamon Museum (2 hours)', 'Evening stroll in Alexanderplatz (1 hour)']}, 'Day 3': {'Berlin': ['Visit the East Side Gallery (1 hour)', 'Explore Kreuzberg district (2 hours)', 'Lunch at a street food market (1 hour)', 'Visit the Jewish Museum (2 hours)', 'Dinner and nightlife in Friedrichshain (3 hours)']}, 'Day 4': {'Berlin': ['Visit Charlottenburg Palace (2 hours)', 'Explore the Berlin Zoo (2.5 hours)', 'Lunch in the Tiergarten (1 hour)', 'Shopping at Kurfürstendamm (2 hours)', 'Depart from Berlin to Milan']}}"
print(generated_travel_plan)

{'Day 1': {'Berlin': ['Arrive in Berlin (morning)', 'Check into hotel (midday)', 'Visit Brandenburg Gate (afternoon)', 'Dinner at a local restaurant (evening)']}, 'Day 2': {'Berlin': ['Visit Berlin Wall Memorial (morning)', 'Explore Museum Island (midday)', 'Lunch at a café (afternoon)', 'Visit Pergamon Museum (afternoon)', 'Evening stroll along the Spree River (evening)']}, 'Day 3': {'Berlin': ['Visit Reichstag Building (morning)', 'Explore Tiergarten Park (midday)', 'Lunch in the park (afternoon)', 'Visit Victory Column (afternoon)', 'Dinner in Kreuzberg (evening)']}, 'Day 4': {'Berlin': ['Visit East Side Gallery (morning)', 'Explore Friedrichshain district (midday)', 'Lunch at a local eatery (afternoon)', 'Visit Jewish Museum (afternoon)', 'Attend a local event or concert (evening)']}, 'Day 5': {'Berlin': ['Visit Charlottenburg Palace (morning)', 'Explore the palace gardens (midday)', 'Lunch at a nearby café (afternoon)', 'Visit the Berlin Zoo (afternoon)', 'Dinner in the Wilmersdor

In [10]:
prompt_generation = """Below it is provided a proxy (in JSON format) of the travel plan that the user wants to perform:
{travel_plan}
\nIn the JSON below I provide the best flights available to {destination_place}
{flights_json}
\nPlease, based on this travel plan, select the best flights for the arrival in {destination_place} and the return in {departure_place}.
Here are the user preferences for the travel plan: '{user_preferences}'.
The general parameters to take into account for the best flights should be (if the user has provided her/his preferences, you should assign to the following parameters a secondary priority):
- flights prices
- arrival time at from {destination_place}
- leave time from  {destination_place}
The output must match the following JSON format: 
{{"departure_flight": {{"flight_id": "chosen_flight_id", "reason": "reason for this flight choice"}}, "return_flight": {{"flight_id": "chosen_flight_id", "reason": "reason for this flight choice"}} }}
"""

travel_plan_prompt_generator = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=("You are an helpful travel plan assistant that generates travel plans based on user's requests"
                          "It is important that the assistant respects the requested output formats."
                          "You must only answer with just the answer to the request, avoiding any kind of conversational sentence.")),
        HumanMessagePromptTemplate.from_template(prompt_generation),
    ]
)

openai_agent = ChatOpenAI(**PARAMETERS['gpt_params'])

update_prompt = travel_plan_prompt_generator#| openai_agent | StrOutputParser()

In [11]:
chain_pars = {'departure_place': PARAMETERS['departure_place'],
              'destination_place': PARAMETERS['destination_place'],
              'travel_plan': generated_travel_plan,
              'flights_json': flights_info_json,
              'user_preferences': '',#'I want to go to Berlin in the afternoon and be back in Milan in the morning',
              'flights_table': df_markdown}

#print(update_prompt.invoke(chain_pars).to_messages()[1].content)

In [12]:
generator = travel_plan_prompt_generator| openai_agent | StrOutputParser() | RunnableLambda(from_json_to_text)

chosen_flights = generator.invoke(chain_pars)

print(chosen_flights)

{'departure_flight': {'flight_id': 0, 'reason': 'Lowest price and early arrival time in Berlin.'}, 'return_flight': {'flight_id': 3, 'reason': 'Lowest price and convenient morning departure from Berlin.'}}


In [13]:
pd.DataFrame(flights_info_json)

,flight_id,departure_time,arrival_time,departure_airport,arrival_airport,airline,price,total_duration
0,0,2024-08-30 06:55,2024-08-30 08:35,Milan Linate Airport,Berlin Brandenburg Airport,easyJet,121,100
1,1,2024-08-30 11:00,2024-08-30 12:40,Milan Linate Airport,Berlin Brandenburg Airport,easyJet,136,100
2,2,2024-08-30 16:55,2024-08-30 18:35,Milan Linate Airport,Berlin Brandenburg Airport,easyJet,176,100
3,3,2024-09-05 08:20,2024-09-05 10:00,Berlin Brandenburg Airport,Milan Linate Airport,easyJet,74,100
4,4,2024-09-05 14:25,2024-09-05 16:10,Berlin Brandenburg Airport,Milan Linate Airport,easyJet,91,105
5,5,2024-09-05 21:10,2024-09-05 22:50,Berlin Brandenburg Airport,Milan Linate Airport,easyJet,101,100


In [14]:
departure_flight_properties = flights_info_json[chosen_flights['departure_flight']['flight_id']]
return_flight_properties = flights_info_json[chosen_flights['return_flight']['flight_id']]

departure_flight_properties['reason'] = chosen_flights['departure_flight']['reason']
departure_flight_properties['type'] = 'departure_flight'
return_flight_properties['reason'] = chosen_flights['return_flight']['reason']
return_flight_properties['type'] = 'return_flight'

flights = [departure_flight_properties, return_flight_properties]
flights

[{'flight_id': 0,
  'departure_time': '2024-08-30 06:55',
  'arrival_time': '2024-08-30 08:35',
  'departure_airport': 'Milan Linate Airport',
  'arrival_airport': 'Berlin Brandenburg Airport',
  'airline': 'easyJet',
  'price': 121,
  'total_duration': 100,
  'reason': 'Lowest price and early arrival time in Berlin.',
  'type': 'departure_flight'},
 {'flight_id': 3,
  'departure_time': '2024-09-05 08:20',
  'arrival_time': '2024-09-05 10:00',
  'departure_airport': 'Berlin Brandenburg Airport',
  'arrival_airport': 'Milan Linate Airport',
  'airline': 'easyJet',
  'price': 74,
  'total_duration': 100,
  'reason': 'Lowest price and convenient morning departure from Berlin.',
  'type': 'return_flight'}]

In [15]:
df

,flight_id,departure_time,arrival_time,departure_airport,arrival_airport,airline,price
0,0,2024-08-30 06:55,2024-08-30 08:35,Milan Linate Airport,Berlin Brandenburg Airport,easyJet,121
1,1,2024-08-30 11:00,2024-08-30 12:40,Milan Linate Airport,Berlin Brandenburg Airport,easyJet,136
2,2,2024-08-30 16:55,2024-08-30 18:35,Milan Linate Airport,Berlin Brandenburg Airport,easyJet,176
3,3,2024-09-05 08:20,2024-09-05 10:00,Berlin Brandenburg Airport,Milan Linate Airport,easyJet,74
4,4,2024-09-05 14:25,2024-09-05 16:10,Berlin Brandenburg Airport,Milan Linate Airport,easyJet,91
5,5,2024-09-05 21:10,2024-09-05 22:50,Berlin Brandenburg Airport,Milan Linate Airport,easyJet,101


## Hotels

In [16]:
HOTEL_API_RESPONSE = {
  "search_metadata": {
    "id": "66d21f442da1579f2b637677",
    "status": "Success",
    "json_endpoint": "https://serpapi.com/searches/3aee736846b992dd/66d21f442da1579f2b637677.json",
    "created_at": "2024-08-30 19:36:36 UTC",
    "processed_at": "2024-08-30 19:36:36 UTC",
    "google_hotels_url": "https://www.google.com/_/TravelFrontendUi/data/batchexecute?rpcids=AtySUc&source-path=/travel/search&hl=en&gl=us&rt=c&soc-app=162&soc-platform=1&soc-device=1",
    "raw_html_file": "https://serpapi.com/searches/3aee736846b992dd/66d21f442da1579f2b637677.html",
    "prettify_html_file": "https://serpapi.com/searches/3aee736846b992dd/66d21f442da1579f2b637677.prettify",
    "total_time_taken": 2.26
  },
  "search_parameters": {
    "engine": "google_hotels",
    "q": "Berlin Hotels",
    "gl": "us",
    "hl": "en",
    "currency": "EUR",
    "check_in_date": "2024-08-30",
    "check_out_date": "2024-09-05",
    "adults": 1,
    "children": 0
  },
  "brands": [
    {
      "id": 33,
      "name": "Accor Live Limitless",
      "children": [
        {
          "id": 85,
          "name": "25hours"
        },
        {
          "id": 15,
          "name": "Adagio"
        },
        {
          "id": 21,
          "name": "Ibis"
        },
        {
          "id": 13,
          "name": "Ibis Budget"
        },
        {
          "id": 103,
          "name": "Ibis Styles"
        },
        {
          "id": 91,
          "name": "Mercure"
        },
        {
          "id": 276,
          "name": "Mövenpick"
        },
        {
          "id": 47,
          "name": "Novotel"
        },
        {
          "id": 90,
          "name": "Pullman Hotels and Resorts"
        },
        {
          "id": 273,
          "name": "SO/"
        }
      ]
    },
    {
      "id": 98,
      "name": "B&B Hotels"
    },
    {
      "id": 18,
      "name": "Best Western International",
      "children": [
        {
          "id": 155,
          "name": "Best Western"
        },
        {
          "id": 104,
          "name": "Best Western Plus"
        },
        {
          "id": 105,
          "name": "Best Western Premier"
        }
      ]
    },
    {
      "id": 247,
      "name": "Campanile"
    },
    {
      "id": 20,
      "name": "Choice Hotels",
      "children": [
        {
          "id": 27,
          "name": "Comfort Inn"
        }
      ]
    },
    {
      "id": 245,
      "name": "Golden Tulip"
    },
    {
      "id": 28,
      "name": "Hilton Honors",
      "children": [
        {
          "id": 81,
          "name": "DoubleTree by Hilton"
        },
        {
          "id": 115,
          "name": "Hampton by Hilton"
        },
        {
          "id": 54,
          "name": "Hilton Hotels & Resorts"
        },
        {
          "id": 41,
          "name": "Waldorf Astoria"
        }
      ]
    },
    {
      "id": 37,
      "name": "Hyatt",
      "children": [
        {
          "id": 117,
          "name": "Grand Hyatt"
        },
        {
          "id": 349,
          "name": "JDV by Hyatt"
        }
      ]
    },
    {
      "id": 17,
      "name": "IHG Hotels & Resorts",
      "children": [
        {
          "id": 42,
          "name": "Crowne Plaza"
        },
        {
          "id": 64,
          "name": "Holiday Inn"
        },
        {
          "id": 56,
          "name": "Holiday Inn Express"
        },
        {
          "id": 87,
          "name": "Hotel Indigo"
        },
        {
          "id": 2,
          "name": "InterContinental Hotels & Resorts"
        },
        {
          "id": 297,
          "name": "Regent"
        }
      ]
    },
    {
      "id": 325,
      "name": "Kempinski"
    },
    {
      "id": 46,
      "name": "Marriott Bonvoy",
      "children": [
        {
          "id": 128,
          "name": "AC Hotels"
        },
        {
          "id": 59,
          "name": "Autograph Collection"
        },
        {
          "id": 86,
          "name": "Courtyard by Marriott"
        },
        {
          "id": 26,
          "name": "JW Marriott Hotels"
        },
        {
          "id": 61,
          "name": "Marriott Hotels & Resorts"
        },
        {
          "id": 129,
          "name": "Moxy"
        },
        {
          "id": 3,
          "name": "Ritz-Carlton Hotel Company"
        },
        {
          "id": 12,
          "name": "Sheraton Hotels and Resorts"
        },
        {
          "id": 39,
          "name": "Westin Hotels & Resorts"
        }
      ]
    },
    {
      "id": 174,
      "name": "Melia Hotels International",
      "children": [
        {
          "id": 179,
          "name": "Innside by Melia"
        },
        {
          "id": 178,
          "name": "Melia Hotels & Resorts"
        }
      ]
    },
    {
      "id": 167,
      "name": "Motel One"
    },
    {
      "id": 169,
      "name": "NH Hotel Group",
      "children": [
        {
          "id": 171,
          "name": "NH Collection Hotels"
        },
        {
          "id": 170,
          "name": "NH Hotels"
        },
        {
          "id": 172,
          "name": "NHow Hotels"
        }
      ]
    },
    {
      "id": 70,
      "name": "Premier Inn"
    },
    {
      "id": 252,
      "name": "Premiere Classe"
    },
    {
      "id": 80,
      "name": "Radisson Hotel Group",
      "children": [
        {
          "id": 132,
          "name": "Park Inn by Radisson"
        },
        {
          "id": 45,
          "name": "Park Plaza"
        },
        {
          "id": 264,
          "name": "Radisson Collection"
        },
        {
          "id": 391,
          "name": "Radisson Individuals"
        },
        {
          "id": 133,
          "name": "Radisson Red"
        }
      ]
    },
    {
      "id": 163,
      "name": "RIU Hotels & Resorts",
      "children": [
        {
          "id": 164,
          "name": "Plaza by RIU"
        }
      ]
    },
    {
      "id": 253,
      "name": "Scandic Hotels"
    },
    {
      "id": 319,
      "name": "Steigenberger Hotels and Resorts"
    },
    {
      "id": 53,
      "name": "Wyndham Hotels & Resorts",
      "children": [
        {
          "id": 150,
          "name": "Wyndham"
        },
        {
          "id": 141,
          "name": "Wyndham Garden"
        }
      ]
    }
  ],
  "properties": [
    {
      "type": "hotel",
      "name": "MEININGER Hotel Berlin Mitte Humboldthaus",
      "description": "Hip lodging with private rooms & mixed dorms, plus a lounge/snack bar, a guest kitchen & free Wi-Fi.",
      "link": "https://www.meininger-hotels.com/en/hotels/berlin/hotel-berlin-center/?utm_source=gmb&utm_medium=referral&utm_campaign=BER-OS&utm_content=website",
      "gps_coordinates": {
        "latitude": 52.5249422,
        "longitude": 13.392138899999999
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "11:00 AM",
      "rate_per_night": {
        "lowest": "€28",
        "extracted_lowest": 28,
        "before_taxes_fees": "€25",
        "extracted_before_taxes_fees": 25
      },
      "total_rate": {
        "lowest": "€169",
        "extracted_lowest": 169,
        "before_taxes_fees": "€151",
        "extracted_before_taxes_fees": 151
      },
      "nearby_places": [
        {
          "name": "Bode-Museum",
          "transportations": [
            {
              "type": "Walking",
              "duration": "7 min"
            }
          ]
        },
        {
          "name": "Berlin Central Train Station",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "8 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "39 min"
            },
            {
              "type": "Public transport",
              "duration": "45 min"
            }
          ]
        },
        {
          "name": "Kamala",
          "transportations": [
            {
              "type": "Walking",
              "duration": "1 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipP2kVVx7B6vZcp_SAuOIQ3W_eIaAtxE2kOVvKeh=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipP2kVVx7B6vZcp_SAuOIQ3W_eIaAtxE2kOVvKeh=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNEzJ8nFMClpmtJXjCGDc1LWRX0I-QamU18sgxs=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNEzJ8nFMClpmtJXjCGDc1LWRX0I-QamU18sgxs=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipN8Lqkfx77XD3iIkMoTkWlWDU2ObvIBB88flY_O=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipN8Lqkfx77XD3iIkMoTkWlWDU2ObvIBB88flY_O=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPRxMabWK9ChqMjoxWUcxLzp_y_iNNPkxwspzFk=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPRxMabWK9ChqMjoxWUcxLzp_y_iNNPkxwspzFk=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOUg0ZyffVDUJimx8IotTy0cseSRnJzjzr4zTWw=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOUg0ZyffVDUJimx8IotTy0cseSRnJzjzr4zTWw=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOtICxyWGYSocsw15lJCfl1f0K7dxU1YoMxgmRk=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOtICxyWGYSocsw15lJCfl1f0K7dxU1YoMxgmRk=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMwfUps-CBL3_hQvAwtB_4xIKf7OZ8QprNfO1p1=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMwfUps-CBL3_hQvAwtB_4xIKf7OZ8QprNfO1p1=s10000"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/WMstqEI4XpB_o5lyqwAkVaU6EbHcGKnSFB1DFN1xKW9PPNrFaHPFKswY2yup2hzmHWObwCY_RBpWYg-eeKPppxJZP6XlnkxrShpPKTou57VJtn1Pw27XfGFeWpK0sQFuGsdjwH06sP1YyR6IYEY2J8zFcfI0He8=s287-w287-h192-n-k-no-v1",
          "original_image": "https://dynamic-media-cdn.tripadvisor.com/media/photo-o/09/21/84/8a/meininger-hotel-berlin.jpg?w=2100&h=1400&s=1"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMkxsf3L4FfZu1Cb6jVGn0eFKczD99RL3TTPeQw=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMkxsf3L4FfZu1Cb6jVGn0eFKczD99RL3TTPeQw=s10000"
        }
      ],
      "overall_rating": 4,
      "reviews": 2914,
      "ratings": [
        {
          "stars": 5,
          "count": 1319
        },
        {
          "stars": 4,
          "count": 894
        },
        {
          "stars": 3,
          "count": 351
        },
        {
          "stars": 2,
          "count": 143
        },
        {
          "stars": 1,
          "count": 207
        }
      ],
      "location_rating": 5.0,
      "reviews_breakdown": [
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 729,
          "positive": 644,
          "negative": 28,
          "neutral": 57
        },
        {
          "name": "Sleep",
          "description": "Sleep",
          "total_mentioned": 438,
          "positive": 129,
          "negative": 255,
          "neutral": 54
        },
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 348,
          "positive": 217,
          "negative": 83,
          "neutral": 48
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 721,
          "positive": 565,
          "negative": 116,
          "neutral": 40
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 857,
          "positive": 566,
          "negative": 205,
          "neutral": 86
        },
        {
          "name": "Family",
          "description": "Family friendly",
          "total_mentioned": 108,
          "positive": 60,
          "negative": 28,
          "neutral": 20
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Parking ($)",
        "Pet-friendly",
        "Bar",
        "Accessible",
        "Child-friendly",
        "Smoke-free property"
      ],
      "property_token": "ChgIqrGY09KrsoPSARoLL2cvMXRmNjkyM3oQAQ",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChgIqrGY09KrsoPSARoLL2cvMXRmNjkyM3oQAQ&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Abba Berlin Hotel",
      "description": "Contemporary lodging with free Wi-Fi, plus a breakfast option, fine dining & an exercise room.",
      "link": "https://www.abbahoteles.com/de/destinationen/abba-berlin-hotel/hotel.html",
      "gps_coordinates": {
        "latitude": 52.4996602,
        "longitude": 13.3216425
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "€78",
        "extracted_lowest": 78,
        "before_taxes_fees": "€70",
        "extracted_before_taxes_fees": 70
      },
      "total_rate": {
        "lowest": "€469",
        "extracted_lowest": 469,
        "before_taxes_fees": "€422",
        "extracted_before_taxes_fees": 422
      },
      "deal": "35% less than usual",
      "deal_description": "Great Deal",
      "nearby_places": [
        {
          "name": "Berlin Zoological Garden",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "6 min"
            }
          ]
        },
        {
          "name": "Berlin Zoologischer Garten Bahnhof",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "5 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "27 min"
            },
            {
              "type": "Public transport",
              "duration": "1 hr"
            }
          ]
        },
        {
          "name": "Capone Restaurant",
          "transportations": [
            {
              "type": "Walking",
              "duration": "4 min"
            }
          ]
        }
      ],
      "hotel_class": "4-star hotel",
      "extracted_hotel_class": 4,
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipN-SIpNPNgXIvZs4YAOGfHxZHPTHwk_lJMKvtm8=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipN-SIpNPNgXIvZs4YAOGfHxZHPTHwk_lJMKvtm8=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPEyoPklrUecTJu9TYfRNmU2YRpEJGi-JIYQ4a9=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPEyoPklrUecTJu9TYfRNmU2YRpEJGi-JIYQ4a9=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMK4zwcIotbN1XiBGtYbhQcH1wy9A2OoFwlCJLY=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMK4zwcIotbN1XiBGtYbhQcH1wy9A2OoFwlCJLY=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMMDrSFeb4ufPYuuJeJtEB1ifhyOkGSiFvivNmr=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMMDrSFeb4ufPYuuJeJtEB1ifhyOkGSiFvivNmr=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOayS6VRe6IR0PUe0BvJtfR3ifQ6-7Til8Ymert=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOayS6VRe6IR0PUe0BvJtfR3ifQ6-7Til8Ymert=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPYiX_j1N11XmFQxIyqc1Hsfes0lvfMkyBmJ9v-=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPYiX_j1N11XmFQxIyqc1Hsfes0lvfMkyBmJ9v-=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOuOytUot8m4jmJTkSi3DoQUTxQQ46kTgFS2EEt=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOuOytUot8m4jmJTkSi3DoQUTxQQ46kTgFS2EEt=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMDssGH6e7rh7SHSFUzJI88MQjv45BxxXY94wI1=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMDssGH6e7rh7SHSFUzJI88MQjv45BxxXY94wI1=s10000"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/HzMt6UwPxZzAYUgs68yu6u3BbVlCEoY7d95119xP_xP8FMiYVcHp-svHSmXa3MBCR8UaREEmZShC2Oymy3fiv-Sb6FQ2qRRmRdjkqGKJbA-aYjDDH6k0oSZz-F5hwmsaHYPygvkQTQZxVE-PYmZPRTMh6G6YTzY=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/135/560/891/69884723_4K_O.jpg"
        }
      ],
      "overall_rating": 4.3,
      "reviews": 2212,
      "ratings": [
        {
          "stars": 5,
          "count": 1199
        },
        {
          "stars": 4,
          "count": 706
        },
        {
          "stars": 3,
          "count": 190
        },
        {
          "stars": 2,
          "count": 56
        },
        {
          "stars": 1,
          "count": 61
        }
      ],
      "location_rating": 4.8,
      "reviews_breakdown": [
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 354,
          "positive": 295,
          "negative": 42,
          "neutral": 17
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 459,
          "positive": 383,
          "negative": 48,
          "neutral": 28
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 567,
          "positive": 488,
          "negative": 52,
          "neutral": 27
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 402,
          "positive": 353,
          "negative": 13,
          "neutral": 36
        },
        {
          "name": "Restaurant",
          "description": "Restaurant",
          "total_mentioned": 174,
          "positive": 141,
          "negative": 25,
          "neutral": 8
        },
        {
          "name": "Wellness",
          "description": "Wellness",
          "total_mentioned": 31,
          "positive": 18,
          "negative": 10,
          "neutral": 3
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Parking ($)",
        "Air conditioning",
        "Pet-friendly",
        "Fitness centre",
        "Spa",
        "Bar",
        "Restaurant",
        "Room service",
        "Full-service laundry",
        "Accessible",
        "Business centre",
        "Child-friendly",
        "Smoke-free property"
      ],
      "eco_certified": True,
      "property_token": "ChcImuiflraNuZMXGgsvZy8xdGN5ZGg5MxAB",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChcImuiflraNuZMXGgsvZy8xdGN5ZGg5MxAB&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "MEININGER Hotel Berlin East Side Gallery",
      "description": "Streamlined dorms & private rooms in a hip budget hotel offering a bar, a lounge & a game area.",
      "link": "https://www.meininger-hotels.com/en/hotels/berlin/hotel-berlin-east-side-gallery/?utm_source=gmb&utm_medium=referral&utm_campaign=BER-AP&utm_content=website",
      "gps_coordinates": {
        "latitude": 52.508320000000005,
        "longitude": 13.4359836
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "11:00 AM",
      "rate_per_night": {
        "lowest": "€42",
        "extracted_lowest": 42,
        "before_taxes_fees": "€37",
        "extracted_before_taxes_fees": 37
      },
      "total_rate": {
        "lowest": "€251",
        "extracted_lowest": 251,
        "before_taxes_fees": "€224",
        "extracted_before_taxes_fees": 224
      },
      "nearby_places": [
        {
          "name": "East Side Gallery",
          "transportations": [
            {
              "type": "Walking",
              "duration": "2 min"
            }
          ]
        },
        {
          "name": "Berlin Ostbahnhof",
          "transportations": [
            {
              "type": "Walking",
              "duration": "4 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Public transport",
              "duration": "30 min"
            },
            {
              "type": "Taxi",
              "duration": "32 min"
            }
          ]
        },
        {
          "name": "Spreewirtschaft Restaurant",
          "transportations": [
            {
              "type": "Walking",
              "duration": "9 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipO5NHO7OTpIMNXlwP0wWl8tvsyVVn5SrmbW2j_p=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipO5NHO7OTpIMNXlwP0wWl8tvsyVVn5SrmbW2j_p=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMDTey3g6iI5HGlLgn0RHXaGVOIH5qEuM6fWZFn=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMDTey3g6iI5HGlLgn0RHXaGVOIH5qEuM6fWZFn=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOOt7IxTGT3qyp3j07o0-WqgSBv-vEXeZy69AHh=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOOt7IxTGT3qyp3j07o0-WqgSBv-vEXeZy69AHh=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOnF1F_AYwGJUyl2MUSdKwYHiPjp5BNyKUYu2dr=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOnF1F_AYwGJUyl2MUSdKwYHiPjp5BNyKUYu2dr=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOM0DbywGgT0VOjca66ok4m4obm5OGEHA8rPbgk=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOM0DbywGgT0VOjca66ok4m4obm5OGEHA8rPbgk=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMhvSQMIBG74KHhAcBZsVrRrXTqqmBP2_hA6N9o=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMhvSQMIBG74KHhAcBZsVrRrXTqqmBP2_hA6N9o=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNMxAmJME8Ofq7_Lfe7CN84N5EffFjwWkIiLWDx=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNMxAmJME8Ofq7_Lfe7CN84N5EffFjwWkIiLWDx=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMmTYFARu0pmSRYV6AqyQPJrVhfMLdvsfrc7ua_=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMmTYFARu0pmSRYV6AqyQPJrVhfMLdvsfrc7ua_=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPwtb_UZoCAmuzbEr_p1SbXLl5al9fztqjnvkT_=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPwtb_UZoCAmuzbEr_p1SbXLl5al9fztqjnvkT_=s10000"
        }
      ],
      "overall_rating": 3.9,
      "reviews": 3724,
      "ratings": [
        {
          "stars": 5,
          "count": 1589
        },
        {
          "stars": 4,
          "count": 1151
        },
        {
          "stars": 3,
          "count": 446
        },
        {
          "stars": 2,
          "count": 203
        },
        {
          "stars": 1,
          "count": 335
        }
      ],
      "location_rating": 4.0,
      "reviews_breakdown": [
        {
          "name": "Sleep",
          "description": "Sleep",
          "total_mentioned": 503,
          "positive": 159,
          "negative": 281,
          "neutral": 63
        },
        {
          "name": "Cleanliness",
          "description": "Cleanliness",
          "total_mentioned": 885,
          "positive": 506,
          "negative": 323,
          "neutral": 56
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 1104,
          "positive": 660,
          "negative": 310,
          "neutral": 134
        },
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 417,
          "positive": 216,
          "negative": 142,
          "neutral": 59
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 747,
          "positive": 661,
          "negative": 28,
          "neutral": 58
        },
        {
          "name": "Bathroom",
          "description": "Bathroom and toiletries",
          "total_mentioned": 453,
          "positive": 144,
          "negative": 255,
          "neutral": 54
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Parking ($)",
        "Pet-friendly",
        "Bar",
        "Kitchen in rooms",
        "Accessible",
        "Child-friendly",
        "Smoke-free property"
      ],
      "property_token": "ChoI3Yiz__eFt_zVARoNL2cvMTFmeF80cmtwZBAB",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChoI3Yiz__eFt_zVARoNL2cvMTFmeF80cmtwZBAB&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Premier Inn Berlin Alexanderplatz hotel",
      "description": "Unpretentious rooms, some with sofabeds, in a low-key hotel featuring a relaxed bar & Wi-Fi.",
      "link": "https://www.premierinn.com/de/de/hotels/deutschland/berlin/berlin/berlin-alexanderplatz.html?cid=GLBC_BERALX",
      "gps_coordinates": {
        "latitude": 52.5237127,
        "longitude": 13.417499699999999
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "€108",
        "extracted_lowest": 108,
        "before_taxes_fees": "€96",
        "extracted_before_taxes_fees": 96
      },
      "total_rate": {
        "lowest": "€650",
        "extracted_lowest": 650,
        "before_taxes_fees": "€579",
        "extracted_before_taxes_fees": 579
      },
      "deal": "15% less than usual",
      "deal_description": "Deal",
      "nearby_places": [
        {
          "name": "Alexanderplatz",
          "transportations": [
            {
              "type": "Walking",
              "duration": "3 min"
            }
          ]
        },
        {
          "name": "Alexanderplatz",
          "transportations": [
            {
              "type": "Walking",
              "duration": "8 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Public transport",
              "duration": "34 min"
            },
            {
              "type": "Taxi",
              "duration": "36 min"
            }
          ]
        },
        {
          "name": "KIM999 Vegan",
          "transportations": [
            {
              "type": "Walking",
              "duration": "6 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMC626sbsuJIKRl5QfkVpPGbrKFLPmCjKxsU_tT=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMC626sbsuJIKRl5QfkVpPGbrKFLPmCjKxsU_tT=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOYiRphTnG4a06WmYYLcEwU-HOS9qeaJ3aR-K38=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOYiRphTnG4a06WmYYLcEwU-HOS9qeaJ3aR-K38=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMcD0Cf7qqTEkrSh6sRYp2UE0850OeIbUifRGk4=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMcD0Cf7qqTEkrSh6sRYp2UE0850OeIbUifRGk4=s10000"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/_Z14hsbl5q4Vht2Ruh-2n5GIw5K_ob1ZmDlsVkPmN9H47FeONuPjLzp6JN6LmQbpFkLDXAtmDN3OQszYgvL03xDwoHln23l0y24UjSLZ5OEJ3BFCIXSvjv7Chb5aqlU_UmIuJEPLXh10_HEKyTO0-_C9aneLBGI=s287-w287-h192-n-k-no-v1",
          "original_image": "https://i.giatamedia.com/m.php?m=AQABAAAAla4KAD7XZAIFADjLxud80y5BfgGceTXdma4"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipN7T2JDhefkc9__Q9Ch83M-7weHfsyGwcBimUl5=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipN7T2JDhefkc9__Q9Ch83M-7weHfsyGwcBimUl5=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPix1E3zt1dSx7A3v0ZccmVRW1HSSEsVphObtW2=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPix1E3zt1dSx7A3v0ZccmVRW1HSSEsVphObtW2=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipM0qQlYGttdAFgekngCgRNHaKajj2Ce9VsJezZU=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipM0qQlYGttdAFgekngCgRNHaKajj2Ce9VsJezZU=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPl6H-RGjWeJYBCtx-e1wCAy_LLp5-KS7Tdsw2l=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPl6H-RGjWeJYBCtx-e1wCAy_LLp5-KS7Tdsw2l=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/DMODHun6C4vn2Uq56yaxr97VgLZUHSFOTG6KGr0ZiyKKclbvZ8bQP3nnzpriHmXATTNY8EkgKVmR2H0Ya1aS9mrffQqx8cxpZ52mPnsooOXETytZ7NFNJpH-g1DIOOYVMR6p9xUf5jglf9KNM6NVw4A15m2-ZQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://i.giatamedia.com/m.php?m=AQABAAAAla4KAHLXZAIFAHM-ma8MeDzq9U1Smj4hBc8"
        }
      ],
      "overall_rating": 4.1,
      "reviews": 1133,
      "ratings": [
        {
          "stars": 5,
          "count": 592
        },
        {
          "stars": 4,
          "count": 275
        },
        {
          "stars": 3,
          "count": 102
        },
        {
          "stars": 2,
          "count": 63
        },
        {
          "stars": 1,
          "count": 101
        }
      ],
      "location_rating": 4.4,
      "reviews_breakdown": [
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 226,
          "positive": 149,
          "negative": 43,
          "neutral": 34
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 371,
          "positive": 243,
          "negative": 112,
          "neutral": 16
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 330,
          "positive": 283,
          "negative": 27,
          "neutral": 20
        },
        {
          "name": "Sleep",
          "description": "Sleep",
          "total_mentioned": 188,
          "positive": 85,
          "negative": 78,
          "neutral": 25
        },
        {
          "name": "Bathroom",
          "description": "Bathroom and toiletries",
          "total_mentioned": 170,
          "positive": 45,
          "negative": 101,
          "neutral": 24
        },
        {
          "name": "Air Conditioning",
          "description": "Air conditioning",
          "total_mentioned": 63,
          "positive": 9,
          "negative": 49,
          "neutral": 5
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Parking",
        "Bar",
        "Restaurant",
        "Accessible",
        "Child-friendly",
        "Smoke-free property"
      ],
      "property_token": "ChkI9rmNhYGcxIN6Gg0vZy8xMXN3em1zbTd4EAE",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChkI9rmNhYGcxIN6Gg0vZy8xMXN3em1zbTd4EAE&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Generator Berlin Mitte",
      "description": "Stylish rooms & dorms in a trendy hostel offering a 24/7 lounge, a cafe/bar & regular social events.",
      "link": "https://staygenerator.com/hostels/berlin/mitte?utm_source=google-my-business&utm_medium=organic&utm_campaign=hostel-Mitte",
      "gps_coordinates": {
        "latitude": 52.525197999999996,
        "longitude": 13.3913823
      },
      "check_in_time": "2:00 PM",
      "check_out_time": "10:00 AM",
      "rate_per_night": {
        "lowest": "€30",
        "extracted_lowest": 30,
        "before_taxes_fees": "€30",
        "extracted_before_taxes_fees": 30
      },
      "total_rate": {
        "lowest": "€178",
        "extracted_lowest": 178,
        "before_taxes_fees": "€178",
        "extracted_before_taxes_fees": 178
      },
      "nearby_places": [
        {
          "name": "Bode-Museum",
          "transportations": [
            {
              "type": "Walking",
              "duration": "7 min"
            }
          ]
        },
        {
          "name": "Berlin Central Train Station",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "8 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "38 min"
            },
            {
              "type": "Public transport",
              "duration": "45 min"
            }
          ]
        },
        {
          "name": "NamPan Restaurant",
          "transportations": [
            {
              "type": "Walking",
              "duration": "1 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMHTyL5cl5I1TO7EskvxC7XUC-skAM6enS7v6gb=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMHTyL5cl5I1TO7EskvxC7XUC-skAM6enS7v6gb=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipO-Szvgv99I-PJcDNlbCEQZTjmJKCkFpid_MGlv=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipO-Szvgv99I-PJcDNlbCEQZTjmJKCkFpid_MGlv=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipN0f5UG9ozp5hl36SyRWluSU60pcHhKIlbvsxo_=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipN0f5UG9ozp5hl36SyRWluSU60pcHhKIlbvsxo_=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNePPNFZmu0zUG0B6X-ThBJGt8xXlBecSW9ldr6=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNePPNFZmu0zUG0B6X-ThBJGt8xXlBecSW9ldr6=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMrzhIa79Fb61FEp2LWKAty1EsYxyR_wFN4zJvi=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMrzhIa79Fb61FEp2LWKAty1EsYxyR_wFN4zJvi=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMNJmhKh5dlDY7ZbLskE5Q_lmFUBcnFOhl-59IG=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMNJmhKh5dlDY7ZbLskE5Q_lmFUBcnFOhl-59IG=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipP7Xj-O6a4HXmLEvqALkYuUY1DMToMv3PNSPqki=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipP7Xj-O6a4HXmLEvqALkYuUY1DMToMv3PNSPqki=s10000"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/Y6PNsQF8fPSHvSFnXVt0qd2kuhtRBjUgB7_ysCJx0WLanNcK3CmDNxXS1IY8kB4MTtKC4abCZAX4N1NYtTApKZscsOB0zgGacfBz6Q4wncRmq-2hI-2678TYPE2_51QpaEsIPTEZtOS4qIrl0wd6XsX1ehSiRw=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/2/0/65/986/802/generator_berlin_mitte22_R.jpg"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/jgNAWSdthP8z6z0dyNWaebIaR-4iXNSHivDLPEWJ39v49PKQkcjEg03xsshciZaWo2rVWC1dP18jC2D6QNcyJPbF8Zw4jpg8mNd3icp9WfRuw6f1nhyalEUWWQhdAgAHwItrrNAAat3xDRKfVgUzzbZeoSpnfA=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/2/0/65/986/466/generator_berlin_mitte2_R.jpg"
        }
      ],
      "overall_rating": 4,
      "reviews": 4202,
      "ratings": [
        {
          "stars": 5,
          "count": 2137
        },
        {
          "stars": 4,
          "count": 1012
        },
        {
          "stars": 3,
          "count": 428
        },
        {
          "stars": 2,
          "count": 245
        },
        {
          "stars": 1,
          "count": 380
        }
      ],
      "location_rating": 5.0,
      "reviews_breakdown": [
        {
          "name": "Bar",
          "description": "Bar or lounge",
          "total_mentioned": 597,
          "positive": 518,
          "negative": 54,
          "neutral": 25
        },
        {
          "name": "Dining",
          "description": "Food and Beverage",
          "total_mentioned": 449,
          "positive": 372,
          "negative": 59,
          "neutral": 18
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 1057,
          "positive": 804,
          "negative": 204,
          "neutral": 49
        },
        {
          "name": "Sleep",
          "description": "Sleep",
          "total_mentioned": 443,
          "positive": 106,
          "negative": 269,
          "neutral": 68
        },
        {
          "name": "Nightlife",
          "description": "Nightlife",
          "total_mentioned": 109,
          "positive": 55,
          "negative": 40,
          "neutral": 14
        },
        {
          "name": "Bathroom",
          "description": "Bathroom and toiletries",
          "total_mentioned": 377,
          "positive": 73,
          "negative": 266,
          "neutral": 38
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Pet-friendly",
        "Bar",
        "Restaurant",
        "Child-friendly"
      ],
      "eco_certified": True,
      "property_token": "ChgIlLbwmcfbkq83GgwvZy8xMmhtZmx6NzkQAQ",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChgIlLbwmcfbkq83GgwvZy8xMmhtZmx6NzkQAQ&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "AC Hotel Berlin Humboldthain Park",
      "description": "Streamlined quarters in a relaxed hotel offering a sleek restaurant with a cocktail bar & a terrace.",
      "link": "https://www.marriott.com/en-us/hotels/berac-ac-hotel-berlin-humboldthain-park/overview/?scid=f2ae0541-1279-4f24-b197-a979c79310b0",
      "gps_coordinates": {
        "latitude": 52.5494944,
        "longitude": 13.384339800000001
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "€131",
        "extracted_lowest": 131,
        "before_taxes_fees": "€117",
        "extracted_before_taxes_fees": 117
      },
      "total_rate": {
        "lowest": "€787",
        "extracted_lowest": 787,
        "before_taxes_fees": "€701",
        "extracted_before_taxes_fees": 701
      },
      "deal": "15% less than usual",
      "deal_description": "Deal",
      "nearby_places": [
        {
          "name": "Berlin Wall Memorial",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "4 min"
            },
            {
              "type": "Public transport",
              "duration": "10 min"
            }
          ]
        },
        {
          "name": "U Pankstraße",
          "transportations": [
            {
              "type": "Walking",
              "duration": "7 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Public transport",
              "duration": "34 min"
            },
            {
              "type": "Taxi",
              "duration": "39 min"
            }
          ]
        },
        {
          "name": "AC Lounge",
          "transportations": [
            {
              "type": "Walking",
              "duration": "1 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/6AnW1pzoqvikWtTBYCvzJAGPhAlfQh5GShzfs7BgNOqGwMWHQ91cMDAHiCUr2YT-4l_r-IgPquQTDDvEzfnl7mRizQCMoiH7bzvTxYvKc2A6Dbm5ZMFCphFKMDWlphPgNllq3VirgeLzWgnaszDKPFoLb1eL3Q=s287-w287-h192-n-k-no-v1",
          "original_image": "https://images.trvl-media.com/lodging/31000000/30570000/30569900/30569864/c3105b3e_z.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPLSZGr5rhsfXGmZI2XpEgZP5svKGqsDYCamtpi=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPLSZGr5rhsfXGmZI2XpEgZP5svKGqsDYCamtpi=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMu2n8cbFfk6dLN_ARu7cQknR7NmLbJitI355q4=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMu2n8cbFfk6dLN_ARu7cQknR7NmLbJitI355q4=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOleQcjyoKSnsKu5liGhFWweaEZBKg_o28U-nae=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOleQcjyoKSnsKu5liGhFWweaEZBKg_o28U-nae=s10000"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/4T_HYPGMk1SKrS42qCqBkWzCLEJCXbgAPRg4hXGqD62q_DNF4PRhBQu39KdxCfM6HjGtYayQ7D-GMs_mrHox5wd1k8RoR90NFBR-yEjQ-7ofHwu9y7isXVxkVFaKZzhpjbhAD9_p8EcQbN3LI9-3MPIhT_sPmg=s287-w287-h192-n-k-no-v1",
          "original_image": "https://images.trvl-media.com/lodging/31000000/30570000/30569900/30569864/f7d878fe_z.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipM672qldY8rmm4oaSCJo8gCHXr0PeuLIGNZ9y4S=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipM672qldY8rmm4oaSCJo8gCHXr0PeuLIGNZ9y4S=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNCHYSAkif-w_7U_gGvpRaN3CFMY2JG1ft7ePjb=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNCHYSAkif-w_7U_gGvpRaN3CFMY2JG1ft7ePjb=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOgikMmVBsCKrilL-SVrd3__RScp689YuSwry6p=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOgikMmVBsCKrilL-SVrd3__RScp689YuSwry6p=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPfGWTSgwVdZyN3JdyPgj47ZDHf1Xd0FimAaJJB=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPfGWTSgwVdZyN3JdyPgj47ZDHf1Xd0FimAaJJB=s10000"
        }
      ],
      "overall_rating": 4.3,
      "reviews": 718,
      "ratings": [
        {
          "stars": 5,
          "count": 430
        },
        {
          "stars": 4,
          "count": 172
        },
        {
          "stars": 3,
          "count": 51
        },
        {
          "stars": 2,
          "count": 24
        },
        {
          "stars": 1,
          "count": 41
        }
      ],
      "location_rating": 4.2,
      "reviews_breakdown": [
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 240,
          "positive": 185,
          "negative": 42,
          "neutral": 13
        },
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 117,
          "positive": 90,
          "negative": 18,
          "neutral": 9
        },
        {
          "name": "Transit",
          "description": "Public transit",
          "total_mentioned": 83,
          "positive": 64,
          "negative": 7,
          "neutral": 12
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 213,
          "positive": 161,
          "negative": 35,
          "neutral": 17
        },
        {
          "name": "Bar",
          "description": "Bar or lounge",
          "total_mentioned": 48,
          "positive": 37,
          "negative": 5,
          "neutral": 6
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 129,
          "positive": 99,
          "negative": 12,
          "neutral": 18
        }
      ],
      "amenities": [
        "Free Wi-Fi",
        "Parking ($)",
        "Air conditioning",
        "Pet-friendly",
        "Bar",
        "Restaurant",
        "Full-service laundry",
        "Accessible",
        "Business centre",
        "Child-friendly",
        "Smoke-free property"
      ],
      "property_token": "ChoIm6_xuISKjs38ARoNL2cvMTFmM3IycDcyMBAB",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChoIm6_xuISKjs38ARoNL2cvMTFmM3IycDcyMBAB&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Novotel Suites Berlin City Potsdamer Platz",
      "description": "Colorful suites in a modern hotel with free Wi-Fi, plus a regional restaurant & a cocktail bar.",
      "link": "https://all.accor.com/lien_externe.svlt?goto=fiche_hotel&code_hotel=3745&merchantid=seo-maps-DE-3745&sourceid=aw-cen&utm_medium=seo%20maps&utm_source=google%20Maps&utm_campaign=seo%20maps",
      "gps_coordinates": {
        "latitude": 52.5045517,
        "longitude": 13.383297299999999
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "€121",
        "extracted_lowest": 121,
        "before_taxes_fees": "€108",
        "extracted_before_taxes_fees": 108
      },
      "total_rate": {
        "lowest": "€726",
        "extracted_lowest": 726,
        "before_taxes_fees": "€648",
        "extracted_before_taxes_fees": 648
      },
      "deal": "19% less than usual",
      "deal_description": "Deal",
      "nearby_places": [
        {
          "name": "Topography of Terror",
          "transportations": [
            {
              "type": "Walking",
              "duration": "6 min"
            }
          ]
        },
        {
          "name": "Berlin Central Train Station",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "10 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "30 min"
            },
            {
              "type": "Public transport",
              "duration": "37 min"
            }
          ]
        },
        {
          "name": "Naina Restaurant Berlin",
          "transportations": [
            {
              "type": "Walking",
              "duration": "1 min"
            }
          ]
        }
      ],
      "hotel_class": "3-star hotel",
      "extracted_hotel_class": 3,
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOT9FSAGp9lDt_Ay_xx7fNJnl3pedMuM1iAz0kV=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOT9FSAGp9lDt_Ay_xx7fNJnl3pedMuM1iAz0kV=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNQ9GrapDWyaV-55PZssy-j37ejsqQZ4onXdoLj=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNQ9GrapDWyaV-55PZssy-j37ejsqQZ4onXdoLj=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOOpdoD8uZ2HqY9exuGrw_esz-S6EL9kNj88npi=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOOpdoD8uZ2HqY9exuGrw_esz-S6EL9kNj88npi=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipO8OYAF491epxkRbMRG-QsWtefTMO9ztDa0PTt-=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipO8OYAF491epxkRbMRG-QsWtefTMO9ztDa0PTt-=s10000"
        },
        {
          "thumbnail": "https://streetviewpixels-pa.googleapis.com/v1/thumbnail?panoid=Dj9BhSK4F3AAAAQvOy3XPw&cb_client=search.estubs.gps&w=287&h=192&yaw=100&pitch=-20&thumbfov=100&scale=2",
          "original_image": "https://lh5.googleusercontent.com/p/Dj9BhSK4F3AAAAQvOy3XPw=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPEiLb1KygKIKC1vTrdsNNh7FUkS0p4efte9ZKi=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPEiLb1KygKIKC1vTrdsNNh7FUkS0p4efte9ZKi=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOyhHwwMXQ47iDrlq3KXrmqd-uHDf8ZC-047v-q=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOyhHwwMXQ47iDrlq3KXrmqd-uHDf8ZC-047v-q=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPJmsquSrft0GNYQTWEZk7vvNyJ5UXM0G2EzAUD=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPJmsquSrft0GNYQTWEZk7vvNyJ5UXM0G2EzAUD=s10000"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/0eWfoX2yHIhEg2m25lHAY_9wKwuispPAumQiGRY0ToSwstg3gNN8uI_0ZNx5BmL5Atw5scRfUQ5jAGDaMxrkAqeAdhVPFwXIrbRcHAcYzEi7HZyz9eWp695T4sH4A0RHKXY6CB6zlOb0Wm_hWT2NwQSXRwzdKw=s287-w287-h192-n-k-no-v1",
          "original_image": "https://i.giatamedia.com/m.php?m=AQABAAAAla4KAHcDrgYFAPv1Jjk5bIjxAkGdnInSeaE"
        }
      ],
      "overall_rating": 4.2,
      "reviews": 2988,
      "ratings": [
        {
          "stars": 5,
          "count": 1375
        },
        {
          "stars": 4,
          "count": 1043
        },
        {
          "stars": 3,
          "count": 359
        },
        {
          "stars": 2,
          "count": 102
        },
        {
          "stars": 1,
          "count": 109
        }
      ],
      "location_rating": 4.6,
      "reviews_breakdown": [
        {
          "name": "Family",
          "description": "Family friendly",
          "total_mentioned": 178,
          "positive": 136,
          "negative": 19,
          "neutral": 23
        },
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 416,
          "positive": 211,
          "negative": 149,
          "neutral": 56
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 686,
          "positive": 618,
          "negative": 21,
          "neutral": 47
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 829,
          "positive": 643,
          "negative": 110,
          "neutral": 76
        },
        {
          "name": "Accessibility",
          "description": "Accessibility",
          "total_mentioned": 131,
          "positive": 17,
          "negative": 95,
          "neutral": 19
        },
        {
          "name": "Bathroom",
          "description": "Bathroom and toiletries",
          "total_mentioned": 281,
          "positive": 101,
          "negative": 129,
          "neutral": 51
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Parking ($)",
        "Air conditioning",
        "Pet-friendly",
        "Fitness centre",
        "Bar",
        "Full-service laundry",
        "Accessible",
        "Business centre",
        "Child-friendly",
        "Smoke-free property"
      ],
      "property_token": "ChgI8bjV-MeUyKMcGgwvZy8xaGMxZ3JfaG0QAQ",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChgI8bjV-MeUyKMcGgwvZy8xaGMxZ3JfaG0QAQ&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Hotel MANI by AMANO",
      "description": "Sophisticated downtown locale offering free Wi-Fi, as well as French dining & a trendy bar/lounge.",
      "link": "https://www.amanogroup.com/de/hotels/berlin-mitte-hotel-mani-by-amano?utm_source=google&utm_medium=gmb&utm_campaign=mani-profil&utm_content=button",
      "gps_coordinates": {
        "latitude": 52.5293018,
        "longitude": 13.399652099999999
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "€129",
        "extracted_lowest": 129,
        "before_taxes_fees": "€115",
        "extracted_before_taxes_fees": 115
      },
      "total_rate": {
        "lowest": "€775",
        "extracted_lowest": 775,
        "before_taxes_fees": "€690",
        "extracted_before_taxes_fees": 690
      },
      "deal": "34% less than usual",
      "deal_description": "Great Deal",
      "nearby_places": [
        {
          "name": "Brandenburg Gate",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "8 min"
            }
          ]
        },
        {
          "name": "Rosenthaler Platz",
          "transportations": [
            {
              "type": "Walking",
              "duration": "2 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "38 min"
            },
            {
              "type": "Public transport",
              "duration": "45 min"
            }
          ]
        },
        {
          "name": "Sucre et Sel",
          "transportations": [
            {
              "type": "Walking",
              "duration": "1 min"
            }
          ]
        }
      ],
      "hotel_class": "3-star hotel",
      "extracted_hotel_class": 3,
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMKQa0GNW3sgEh33ZD7vA5c8jRS30-wOcEGZgEN=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMKQa0GNW3sgEh33ZD7vA5c8jRS30-wOcEGZgEN=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNNTBOJ2bELEbo6tNHnnmvpTLFv00kdQwvj69un=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNNTBOJ2bELEbo6tNHnnmvpTLFv00kdQwvj69un=s10000"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/pUFX-ia_UCRJHh-QLxkwXCSFQriq7eLNI5yD_8n-fcecdNSVJ8QJDJpNb1Fu9BrwOWG21pzIIpcAlJ6JpCw_zzpXGC6u6Ljyt9iSs_ssIqwT9wBbLfRK5T3AGTIMN2O6PSk-im09v0-aIC2m6xF83lx-sZOxrA=s287-w287-h192-n-k-no-v1",
          "original_image": "https://www.iceportal.com/data/2813-123694-f79557671_XXLejpg/Germany/Berlin/Hotel-MANI-by-AMANO/Photo/Hotel-MANI-by-AMANO-Berlin-Undefined-MANI-Standard-Room-vision-photos.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMFkkCVTfaAbdvcRpLU2x0zKLtvcVn3VGtC_Gvx=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMFkkCVTfaAbdvcRpLU2x0zKLtvcVn3VGtC_Gvx=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/igHSBCxHKIjUKuq9GZbgittQJEpRlLHXw4pjGigcQiX-XuR2M1L-j3PZ5jXUODRx19-sVeqek8OA79KclQwRuoWhx1v2lTp2KZ37MWVGq_EbcLhBNHsbGhLIHk0Jm9MSyo7MTxsmKbvZo2Jk1sApA_XQezjDwQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://www.iceportal.com/data/2813-123694-f79557685_XXLejpg/Germany/Berlin/Hotel-MANI-by-AMANO/Photo/Hotel-MANI-by-AMANO-Berlin-Undefined-MANI-Restaurant-Bar-Attila-Hartwig.jpg"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/8swk2dwbmC1R1bmaH7GCNvPUVGDncPIcMF6ZHOZizeqhewqv0jSw9hrzuWWV8ebEGe6JCAdO6EQuAz5BTFDFwWvbEsOLUo65Y5TxtTKRvQuToI2KQvo2u3ro2MkcNyxEkNYj8vIfcDwoFYlew9SRcujbuqTP3w=s287-w287-h192-n-k-no-v1",
          "original_image": "https://www.iceportal.com/data/2813-123694-f79557681_XXLejpg/Germany/Berlin/Hotel-MANI-by-AMANO/Photo/Hotel-MANI-by-AMANO-Berlin-Undefined-MANI-Restaurant-vision-photos.jpg"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/uHdA-4YTbuegfZmbxw2Bcd5JyUJCyj4sFUeSmqlH2iYLh-d0EZaEtKu2SeUjj9Iglj4AIWxImbtO0ymjScgZNtMFXJxrVYcpfokq68giWP1H6DescbltmdgJX9ECurpHIaZovln5aesKEq1lfsAoU8s-1pT_8AY=s287-w287-h192-n-k-no-v1",
          "original_image": "https://www.iceportal.com/data/2813-123694-f79557667_XXLejpg/Germany/Berlin/Hotel-MANI-by-AMANO/Photo/Hotel-MANI-by-AMANO-Berlin-Undefined-MANI-Reception-vision-photos.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPQDVcQgimcX-qUeRVY8zMnwwcLJ_PQjUxs8oMi=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPQDVcQgimcX-qUeRVY8zMnwwcLJ_PQjUxs8oMi=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMta9gjBMD1Hzy9x_EDIubWfMiL8_chqKhU4wir=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMta9gjBMD1Hzy9x_EDIubWfMiL8_chqKhU4wir=s10000"
        }
      ],
      "overall_rating": 4.2,
      "reviews": 571,
      "ratings": [
        {
          "stars": 5,
          "count": 321
        },
        {
          "stars": 4,
          "count": 152
        },
        {
          "stars": 3,
          "count": 41
        },
        {
          "stars": 2,
          "count": 25
        },
        {
          "stars": 1,
          "count": 32
        }
      ],
      "location_rating": 5.0,
      "reviews_breakdown": [
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 210,
          "positive": 172,
          "negative": 23,
          "neutral": 15
        },
        {
          "name": "Nightlife",
          "description": "Nightlife",
          "total_mentioned": 25,
          "positive": 4,
          "negative": 15,
          "neutral": 6
        },
        {
          "name": "Dining",
          "description": "Food and Beverage",
          "total_mentioned": 53,
          "positive": 36,
          "negative": 13,
          "neutral": 4
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 126,
          "positive": 119,
          "negative": 6,
          "neutral": 1
        },
        {
          "name": "Atmosphere",
          "description": "Atmosphere",
          "total_mentioned": 73,
          "positive": 68,
          "negative": 3,
          "neutral": 2
        },
        {
          "name": "Sleep",
          "description": "Sleep",
          "total_mentioned": 64,
          "positive": 19,
          "negative": 40,
          "neutral": 5
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Parking ($)",
        "Air conditioning",
        "Pet-friendly",
        "Bar",
        "Restaurant",
        "Airport shuttle",
        "Full-service laundry",
        "Accessible",
        "Child-friendly",
        "Smoke-free property"
      ],
      "property_token": "ChcI8fDmoouOkNwnGgsvZy8xdGR2ajY3MxAB",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChcI8fDmoouOkNwnGgsvZy8xdGR2ajY3MxAB&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "LOGINN Hotel Berlin Airport",
      "link": "https://achat-hotels.com/berlin",
      "gps_coordinates": {
        "latitude": 52.3986037,
        "longitude": 13.541321199999999
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "11:00 AM",
      "rate_per_night": {
        "lowest": "€68",
        "extracted_lowest": 68,
        "before_taxes_fees": "€65",
        "extracted_before_taxes_fees": 65
      },
      "total_rate": {
        "lowest": "€407",
        "extracted_lowest": 407,
        "before_taxes_fees": "€391",
        "extracted_before_taxes_fees": 391
      },
      "deal": "24% less than usual",
      "deal_description": "Deal",
      "nearby_places": [
        {
          "name": "Grünbergallee",
          "transportations": [
            {
              "type": "Walking",
              "duration": "1 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "10 min"
            },
            {
              "type": "Public transport",
              "duration": "14 min"
            }
          ]
        },
        {
          "name": "Schützengrill",
          "transportations": [
            {
              "type": "Walking",
              "duration": "1 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNcVSq5LSfwz2WkfW8B-XUu-VRMHeueLX3LJ4qc=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNcVSq5LSfwz2WkfW8B-XUu-VRMHeueLX3LJ4qc=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNu4NcGLTvqS2dq-rqJp9oThhImTbPItg_-JwzY=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNu4NcGLTvqS2dq-rqJp9oThhImTbPItg_-JwzY=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipObwnsA4Tv09FtbXIynzT9sREiXaDzg9mmuEObT=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipObwnsA4Tv09FtbXIynzT9sREiXaDzg9mmuEObT=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNS6lIUwFKjtacbBuakizpcBs8nwToJEbIwtZyQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNS6lIUwFKjtacbBuakizpcBs8nwToJEbIwtZyQ=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOaaYx6yERY_fkraw_2QpsWpqSWlmsXOu_2sQ_k=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOaaYx6yERY_fkraw_2QpsWpqSWlmsXOu_2sQ_k=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPX9ukkKjFDE7_J_Vz73TqHYTpqi3Ee7lL-xOze=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPX9ukkKjFDE7_J_Vz73TqHYTpqi3Ee7lL-xOze=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNpBZCXSbck52vU6hUBrqCUTBIlFEQgSt9n0mpv=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNpBZCXSbck52vU6hUBrqCUTBIlFEQgSt9n0mpv=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMDE3jfjmsyzGncgw_ODIr8pqbIo-EDA_UkjBDB=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMDE3jfjmsyzGncgw_ODIr8pqbIo-EDA_UkjBDB=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOqCm1o9ymmGyJ0yYGE7rpMCEm6-MEBNoEdc7wI=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOqCm1o9ymmGyJ0yYGE7rpMCEm6-MEBNoEdc7wI=s10000"
        }
      ],
      "overall_rating": 4.5,
      "reviews": 448,
      "ratings": [
        {
          "stars": 5,
          "count": 300
        },
        {
          "stars": 4,
          "count": 97
        },
        {
          "stars": 3,
          "count": 34
        },
        {
          "stars": 2,
          "count": 7
        },
        {
          "stars": 1,
          "count": 10
        }
      ],
      "location_rating": 3.2,
      "reviews_breakdown": [
        {
          "name": "Transit",
          "description": "Public transit",
          "total_mentioned": 72,
          "positive": 53,
          "negative": 5,
          "neutral": 14
        },
        {
          "name": "Bar",
          "description": "Bar or lounge",
          "total_mentioned": 54,
          "positive": 50,
          "negative": 2,
          "neutral": 2
        },
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 80,
          "positive": 63,
          "negative": 11,
          "neutral": 6
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 134,
          "positive": 115,
          "negative": 14,
          "neutral": 5
        },
        {
          "name": "Gym",
          "description": "Gym",
          "total_mentioned": 12,
          "positive": 10,
          "negative": 1,
          "neutral": 1
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 95,
          "positive": 72,
          "negative": 9,
          "neutral": 14
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Parking ($)",
        "Air conditioning",
        "Pet-friendly",
        "Fitness centre",
        "Bar",
        "Kitchen in some rooms",
        "Accessible",
        "Smoke-free property"
      ],
      "eco_certified": True,
      "property_token": "ChoI0LfHr47G-erZARoNL2cvMTF0NDR4Mno5ahAB",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChoI0LfHr47G-erZARoNL2cvMTF0NDR4Mno5ahAB&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Casa Camper Berlin",
      "description": "Trendy hotel with colorful rooms, plus a 24-hour lounge offering complimentary breakfast & snacks.",
      "link": "https://www.casacamper.com/en/berlin/hotel.html",
      "gps_coordinates": {
        "latitude": 52.5257485,
        "longitude": 13.404045799999999
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "€61",
        "extracted_lowest": 61,
        "before_taxes_fees": "€54",
        "extracted_before_taxes_fees": 54
      },
      "total_rate": {
        "lowest": "€364",
        "extracted_lowest": 364,
        "before_taxes_fees": "€325",
        "extracted_before_taxes_fees": 325
      },
      "nearby_places": [
        {
          "name": "Brandenburg Gate",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "8 min"
            }
          ]
        },
        {
          "name": "Weinmeisterstraße",
          "transportations": [
            {
              "type": "Walking",
              "duration": "1 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "39 min"
            },
            {
              "type": "Public transport",
              "duration": "42 min"
            }
          ]
        },
        {
          "name": "Sophien 11",
          "transportations": [
            {
              "type": "Walking",
              "duration": "3 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNh04AEAwJAdawoP-H-4Bz2Nps0WEMiPY_haIMq=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNh04AEAwJAdawoP-H-4Bz2Nps0WEMiPY_haIMq=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMi-3PWQdTlDRSvI38SttU2D-NugTpQO6tiqppx=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMi-3PWQdTlDRSvI38SttU2D-NugTpQO6tiqppx=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNnwiUxL_TOUgYTspd9Ex9WBUrr-02nhY85M0Bi=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNnwiUxL_TOUgYTspd9Ex9WBUrr-02nhY85M0Bi=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipP80Zo6tUrFqdlKegA8eZ0RNxNua7urdtwL6wJK=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipP80Zo6tUrFqdlKegA8eZ0RNxNua7urdtwL6wJK=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNcxjsDlmZ7jK01O5ljDNYJ117t-TFSDxmQVf8_=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNcxjsDlmZ7jK01O5ljDNYJ117t-TFSDxmQVf8_=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMfowBzUrLsmeS7Iwl5XbowS8KZ1aKFfu8qZaT-=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMfowBzUrLsmeS7Iwl5XbowS8KZ1aKFfu8qZaT-=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipO-7Mo0K0NFeS5My92sPZ2FyimdbYHuxs_wG95k=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipO-7Mo0K0NFeS5My92sPZ2FyimdbYHuxs_wG95k=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipParEBNcISll3K1BV-jWy9ImuvaVCinmGOFPenK=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipParEBNcISll3K1BV-jWy9ImuvaVCinmGOFPenK=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOZubjQP2aWw-4btwVg0jR4QYbPVkWPR0kElm3j=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOZubjQP2aWw-4btwVg0jR4QYbPVkWPR0kElm3j=s10000"
        }
      ],
      "overall_rating": 4.7,
      "reviews": 520,
      "ratings": [
        {
          "stars": 5,
          "count": 409
        },
        {
          "stars": 4,
          "count": 74
        },
        {
          "stars": 3,
          "count": 16
        },
        {
          "stars": 2,
          "count": 11
        },
        {
          "stars": 1,
          "count": 10
        }
      ],
      "location_rating": 5.0,
      "reviews_breakdown": [
        {
          "name": "Bar",
          "description": "Bar or lounge",
          "total_mentioned": 96,
          "positive": 83,
          "negative": 7,
          "neutral": 6
        },
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 103,
          "positive": 90,
          "negative": 5,
          "neutral": 8
        },
        {
          "name": "Wellness",
          "description": "Wellness",
          "total_mentioned": 15,
          "positive": 14,
          "negative": 1,
          "neutral": 0
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 134,
          "positive": 123,
          "negative": 2,
          "neutral": 9
        },
        {
          "name": "Dining",
          "description": "Food and Beverage",
          "total_mentioned": 62,
          "positive": 56,
          "negative": 4,
          "neutral": 2
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 180,
          "positive": 163,
          "negative": 6,
          "neutral": 11
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Air conditioning",
        "Fitness centre",
        "Spa",
        "Restaurant",
        "Airport shuttle",
        "Full-service laundry",
        "Accessible",
        "Business centre",
        "Child-friendly",
        "Smoke-free property"
      ],
      "property_token": "ChgI-LqJqpixvc7GARoLL2cvMXRkdmNtOHAQAQ",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChgI-LqJqpixvc7GARoLL2cvMXRkdmNtOHAQAQ&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "St Christopher's Inn Alexanderplatz",
      "description": "Simple dormitories, private rooms & apartments, plus a casual restaurant/bar & free breakfast.",
      "link": "https://www.st-christophers.co.uk/berlin/alexanderplatz-hostel",
      "gps_coordinates": {
        "latitude": 52.5268917,
        "longitude": 13.410606999999999
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "11:00 AM",
      "rate_per_night": {
        "lowest": "€23",
        "extracted_lowest": 23,
        "before_taxes_fees": "€21",
        "extracted_before_taxes_fees": 21
      },
      "total_rate": {
        "lowest": "€140",
        "extracted_lowest": 140,
        "before_taxes_fees": "€124",
        "extracted_before_taxes_fees": 124
      },
      "nearby_places": [
        {
          "name": "Alexanderplatz",
          "transportations": [
            {
              "type": "Walking",
              "duration": "9 min"
            }
          ]
        },
        {
          "name": "Rosa-Luxemburg-Platz",
          "transportations": [
            {
              "type": "Walking",
              "duration": "2 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "39 min"
            },
            {
              "type": "Public transport",
              "duration": "41 min"
            }
          ]
        },
        {
          "name": "BAR internazionale",
          "transportations": [
            {
              "type": "Walking",
              "duration": "1 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOvuPNLuo49kQLNNPKxDHi9hCxIpf5YnaCOrHBD=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOvuPNLuo49kQLNNPKxDHi9hCxIpf5YnaCOrHBD=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPxhrYxYOfZ7YUeYkcx3MneAJD5Q8xS6Et4RXSf=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPxhrYxYOfZ7YUeYkcx3MneAJD5Q8xS6Et4RXSf=s10000"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/vk6Nk3u8VIik5cXNkOhWOBPRa5W_z4xy2Ei-6wdKdoEd_u6fLm6YBUBlJjxWUWE3nGyuxq0ilc89K1yD_h9jlnpCGqS6AFzxUyBPztAPTMEqZLgWvJzKP1PFRgFwZLCi4dS4D5DsmY7gghGUuAPB8xyfKf8rO1M=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/http://media.staticontent.com/media/pictures/cfd75617-756c-4892-8a96-265ca55c0a93=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOIg5b02JKXj3dsaHOpttNmMOjA8h_opl_MhBEE=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOIg5b02JKXj3dsaHOpttNmMOjA8h_opl_MhBEE=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNEHrud8k-ahi1jxyN_bl9fZhE9EJfReB7XTO7a=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNEHrud8k-ahi1jxyN_bl9fZhE9EJfReB7XTO7a=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMagbojh9JPznNVPJ34iKrqQIxohovoUxXrqFZ0=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMagbojh9JPznNVPJ34iKrqQIxohovoUxXrqFZ0=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPkPSLMcmurddNIAVj2_AeRZbPzGVHs4pGAlkIJ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPkPSLMcmurddNIAVj2_AeRZbPzGVHs4pGAlkIJ=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOQtaz_8mufI1THx3sRXeQkvG9NcRg-e6ojhPJU=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOQtaz_8mufI1THx3sRXeQkvG9NcRg-e6ojhPJU=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMjE-PwSN7OIQLU921Fh3etfYsN4MlalXUIt0n7=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMjE-PwSN7OIQLU921Fh3etfYsN4MlalXUIt0n7=s10000"
        }
      ],
      "overall_rating": 4.2,
      "reviews": 2209,
      "ratings": [
        {
          "stars": 5,
          "count": 1217
        },
        {
          "stars": 4,
          "count": 553
        },
        {
          "stars": 3,
          "count": 188
        },
        {
          "stars": 2,
          "count": 100
        },
        {
          "stars": 1,
          "count": 151
        }
      ],
      "location_rating": 4.9,
      "reviews_breakdown": [
        {
          "name": "Bar",
          "description": "Bar or lounge",
          "total_mentioned": 340,
          "positive": 251,
          "negative": 55,
          "neutral": 34
        },
        {
          "name": "Dining",
          "description": "Food and Beverage",
          "total_mentioned": 254,
          "positive": 197,
          "negative": 40,
          "neutral": 17
        },
        {
          "name": "Nightlife",
          "description": "Nightlife",
          "total_mentioned": 80,
          "positive": 50,
          "negative": 21,
          "neutral": 9
        },
        {
          "name": "Sleep",
          "description": "Sleep",
          "total_mentioned": 291,
          "positive": 124,
          "negative": 124,
          "neutral": 43
        },
        {
          "name": "Bathroom",
          "description": "Bathroom and toiletries",
          "total_mentioned": 278,
          "positive": 113,
          "negative": 133,
          "neutral": 32
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 464,
          "positive": 345,
          "negative": 99,
          "neutral": 20
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Bar",
        "Restaurant",
        "Kitchen in rooms"
      ],
      "property_token": "ChgIweLKpfmg_usuGgwvZy8xMnZ0dzVfMWsQAQ",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChgIweLKpfmg_usuGgwvZy8xMnZ0dzVfMWsQAQ&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Sheraton Berlin Grand Hotel Esplanade",
      "description": "Upscale rooms in a modern hotel with a spa, a laid-back restaurant & a refined cocktail bar.",
      "link": "https://www.marriott.com/en-us/hotels/bersi-sheraton-berlin-grand-hotel-esplanade/overview/?scid=f2ae0541-1279-4f24-b197-a979c79310b0",
      "gps_coordinates": {
        "latitude": 52.505323399999995,
        "longitude": 13.3551731
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "11:00 AM",
      "rate_per_night": {
        "lowest": "€119",
        "extracted_lowest": 119,
        "before_taxes_fees": "€106",
        "extracted_before_taxes_fees": 106
      },
      "total_rate": {
        "lowest": "€712",
        "extracted_lowest": 712,
        "before_taxes_fees": "€634",
        "extracted_before_taxes_fees": 634
      },
      "deal": "32% less than usual",
      "deal_description": "Great Deal",
      "nearby_places": [
        {
          "name": "Brandenburg Gate",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "9 min"
            }
          ]
        },
        {
          "name": "Berlin Central Train Station",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "10 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "30 min"
            },
            {
              "type": "Public transport",
              "duration": "1 hr 8 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/Kx1f_xRm_ADXi8Jjz3dSfLxY1s-2SVgYpfoPi-cGfPK53hY-ECI9HedieMTVts0UjJDCbMKgvp6t6X8rl7LVLPzbdbz60gzfR6iWP85r_xdsqMCVPgSUKv_bJDWuyPRN9KgFoGH3M5uM0nioFoVpl4I49kYsrAI=s287-w287-h192-n-k-no-v1",
          "original_image": "https://photos.hotelbeds.com/giata/original/00/005851/005851a_hb_a_029.jpg"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/7F4R6DCIDaIWDsHkJkJFf8lxYw5YxNuONYuQibcZFUnjPuyQokPgqVgCCASpUGA1SRrW04QK0LpXCUIQX_rlnjg1hwhSW2vLzzXqZ544_0azHAd_05mGkJdDe3fY87XwkHr5y9SQkxzs05z7037rjKNsG0ac2F8=s287-w287-h192-n-k-no-v1",
          "original_image": "https://photos.hotelbeds.com/giata/original/00/005851/005851a_hb_a_083.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMeE2bvB48xlIc4pQUUosrmX--3fp6XE38LCTfC=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMeE2bvB48xlIc4pQUUosrmX--3fp6XE38LCTfC=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMnCHi0vTlgZvT34ECsfzGJWA02zbh1vBY8GX-w=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMnCHi0vTlgZvT34ECsfzGJWA02zbh1vBY8GX-w=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOHGGwaui_zYgveps_okzWAsRpmu78Ena8tYvTF=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOHGGwaui_zYgveps_okzWAsRpmu78Ena8tYvTF=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPhvJlkJiLnWk9q42irxU09kscFSn4sdB_IZCeH=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPhvJlkJiLnWk9q42irxU09kscFSn4sdB_IZCeH=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOuvsynigRmKkHU9McHHXO90_yG4G6hIuFJcIQh=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOuvsynigRmKkHU9McHHXO90_yG4G6hIuFJcIQh=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOHKHRRcXa64uKoKplrvC8bxWgfbaATzLXmWvyp=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOHKHRRcXa64uKoKplrvC8bxWgfbaATzLXmWvyp=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipN06Kr0dXp-Sgw0X4Qjdk2JllHTCpufPJhYxsHp=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipN06Kr0dXp-Sgw0X4Qjdk2JllHTCpufPJhYxsHp=s10000"
        }
      ],
      "overall_rating": 4.3,
      "reviews": 2637,
      "ratings": [
        {
          "stars": 5,
          "count": 1471
        },
        {
          "stars": 4,
          "count": 735
        },
        {
          "stars": 3,
          "count": 235
        },
        {
          "stars": 2,
          "count": 101
        },
        {
          "stars": 1,
          "count": 95
        }
      ],
      "location_rating": 4.2,
      "reviews_breakdown": [
        {
          "name": "Fitness",
          "description": "Fitness",
          "total_mentioned": 109,
          "positive": 57,
          "negative": 41,
          "neutral": 11
        },
        {
          "name": "Wellness",
          "description": "Wellness",
          "total_mentioned": 70,
          "positive": 33,
          "negative": 24,
          "neutral": 13
        },
        {
          "name": "Bar",
          "description": "Bar or lounge",
          "total_mentioned": 165,
          "positive": 120,
          "negative": 30,
          "neutral": 15
        },
        {
          "name": "Pool",
          "description": "Pool",
          "total_mentioned": 45,
          "positive": 31,
          "negative": 12,
          "neutral": 2
        },
        {
          "name": "Spa",
          "description": "Spa",
          "total_mentioned": 51,
          "positive": 23,
          "negative": 15,
          "neutral": 13
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 534,
          "positive": 389,
          "negative": 111,
          "neutral": 34
        }
      ],
      "amenities": [
        "Free Wi-Fi",
        "Parking ($)",
        "Air conditioning",
        "Pet-friendly",
        "Fitness centre",
        "Bar",
        "Restaurant",
        "Room service",
        "Accessible",
        "Business centre",
        "Child-friendly",
        "Smoke-free property"
      ],
      "property_token": "ChgIuL77voDM7PMBGgwvZy8xcTZqMGp3NmgQAQ",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChgIuL77voDM7PMBGgwvZy8xcTZqMGp3NmgQAQ&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "The Mandala Hotel",
      "description": "High-end hotel featuring a refined restaurant & a rooftop spa, plus a fitness center.",
      "link": "https://www.themandala.de/",
      "gps_coordinates": {
        "latitude": 52.508980099999995,
        "longitude": 13.373681200000002
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "11:00 AM",
      "rate_per_night": {
        "lowest": "€203",
        "extracted_lowest": 203,
        "before_taxes_fees": "€193",
        "extracted_before_taxes_fees": 193
      },
      "total_rate": {
        "lowest": "€1,215",
        "extracted_lowest": 1215,
        "before_taxes_fees": "€1,157",
        "extracted_before_taxes_fees": 1157
      },
      "deal": "18% less than usual",
      "deal_description": "Deal",
      "nearby_places": [
        {
          "name": "Potsdamer Platz",
          "transportations": [
            {
              "type": "Walking",
              "duration": "3 min"
            }
          ]
        },
        {
          "name": "Potsdamer Platz",
          "transportations": [
            {
              "type": "Walking",
              "duration": "3 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "31 min"
            },
            {
              "type": "Public transport",
              "duration": "43 min"
            }
          ]
        },
        {
          "name": "essenza Potsdamer Platz 1",
          "transportations": [
            {
              "type": "Walking",
              "duration": "2 min"
            }
          ]
        }
      ],
      "hotel_class": "5-star hotel",
      "extracted_hotel_class": 5,
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPXa1z05LN2-VcR9kGAGrg3C81DaA1lctggX-0M=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPXa1z05LN2-VcR9kGAGrg3C81DaA1lctggX-0M=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOabxDeeLrXN_PEtn7QqexUo76Kl1Cq8OyCbtX6=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOabxDeeLrXN_PEtn7QqexUo76Kl1Cq8OyCbtX6=s10000"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/DV4yG_TRwvp2nVRxtadtj_O8kcaZQCj4guzhxyleiZf8LLOiEj2DzYrbl29Ss2ClIuOtpoISv128r7v9TL8B1m_mFU_DpurKkS7ET53BrZdwu-IMiQRRddZVJHGD8gNGrreWonG1JYP6Y68s5vNsEUgqdsSonM8=s287-w287-h192-n-k-no-v1",
          "original_image": "https://photos.hotelbeds.com/giata/original/15/151943/151943a_hb_r_003.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/MvRQZkURMArFseiA4gLssoXwuWi--PLdRr5G2NrIeUNqyY8nRN7pPseRYUYyD7U66MEiH1jGFwJ2bRaLdc0OvMtc-rGWBHbop5TN2Xu7tf__P07m7d6Q6K0Du7jYp8EkaiokIjws1YgM6L_Z8WF1ZWFVHKOIpQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/1/0/176/495/643/40_exterior_view_O.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipN08jmJlaeiHlN6dd3Ddk-rk59cCbKsNf5cCRC2=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipN08jmJlaeiHlN6dd3Ddk-rk59cCbKsNf5cCRC2=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNv6bi7DEaHhHzOXnji2yqbEXkRH8X5oLWtMnZa=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNv6bi7DEaHhHzOXnji2yqbEXkRH8X5oLWtMnZa=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/Db4EbxG6TmPnxxwUMSRTYApLX9vZA30dESDAUpKMv49urkWBpHUa8Ywk54I-_ndU4_rgS9aWc7Wlogt8v3FN-Jx_zQv2rGvM3inkZSprbBY1ZfF6UeWJhyKTU12NJge_Vz6nZ1Xvf6pyWEZOnJwllgGnGjVxhg=s287-w287-h192-n-k-no-v1",
          "original_image": "https://photos.hotelbeds.com/giata/original/15/151943/151943a_hb_r_004.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipMCgWp5jzCY7mhlly1dxgpnIFaTiW-fDegOyyOK=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipMCgWp5jzCY7mhlly1dxgpnIFaTiW-fDegOyyOK=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipM24r9yuVs7pEFEatXfdnU3fHBs34L5ujqIbZ0Q=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipM24r9yuVs7pEFEatXfdnU3fHBs34L5ujqIbZ0Q=s10000"
        }
      ],
      "overall_rating": 4.7,
      "reviews": 593,
      "ratings": [
        {
          "stars": 5,
          "count": 477
        },
        {
          "stars": 4,
          "count": 76
        },
        {
          "stars": 3,
          "count": 31
        },
        {
          "stars": 2,
          "count": 3
        },
        {
          "stars": 1,
          "count": 6
        }
      ],
      "location_rating": 4.9,
      "reviews_breakdown": [
        {
          "name": "Wellness",
          "description": "Wellness",
          "total_mentioned": 36,
          "positive": 25,
          "negative": 6,
          "neutral": 5
        },
        {
          "name": "Spa",
          "description": "Spa",
          "total_mentioned": 29,
          "positive": 22,
          "negative": 3,
          "neutral": 4
        },
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 119,
          "positive": 95,
          "negative": 16,
          "neutral": 8
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 215,
          "positive": 192,
          "negative": 19,
          "neutral": 4
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 226,
          "positive": 213,
          "negative": 9,
          "neutral": 4
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 154,
          "positive": 150,
          "negative": 0,
          "neutral": 4
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Parking",
        "Air conditioning",
        "Fitness centre",
        "Spa",
        "Bar",
        "Restaurant",
        "Room service",
        "Kitchen in rooms",
        "Airport shuttle",
        "Full-service laundry",
        "Accessible",
        "Business centre",
        "Child-friendly",
        "Smoke-free property"
      ],
      "eco_certified": True,
      "property_token": "ChkItojZ06DDv6zHARoML2cvMTE5djhqbXRmEAE",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChkItojZ06DDv6zHARoML2cvMTE5djhqbXRmEAE&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Best Western Plus Plaza Berlin Kurfürstendamm",
      "description": "Relaxed rooms in a contemporary hotel featuring a Spanish steakhouse & a tapas bar, plus free Wi-Fi.",
      "link": "https://plazahotels.de/hotel-berlin/",
      "gps_coordinates": {
        "latitude": 52.5017863,
        "longitude": 13.3226093
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "11:00 AM",
      "rate_per_night": {
        "lowest": "€117",
        "extracted_lowest": 117,
        "before_taxes_fees": "€104",
        "extracted_before_taxes_fees": 104
      },
      "total_rate": {
        "lowest": "€700",
        "extracted_lowest": 700,
        "before_taxes_fees": "€623",
        "extracted_before_taxes_fees": 623
      },
      "deal": "20% less than usual",
      "deal_description": "Deal",
      "nearby_places": [
        {
          "name": "Berlin Zoological Garden",
          "transportations": [
            {
              "type": "Public transport",
              "duration": "9 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "28 min"
            },
            {
              "type": "Public transport",
              "duration": "59 min"
            }
          ]
        },
        {
          "name": "Restaurant El Dorado",
          "transportations": [
            {
              "type": "Walking",
              "duration": "1 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/8V9oWv9DZ_6vj08VVqjgX_4OxoxRzw8OdF5GArQeLoe71bg63nxxLQ44woVg8yE9SDlYNjF12YlXr1oABBpepprCkyT9AENj3MPrUeC1BqGdOpxNhUy9VcswdZytLth6EXdVI_4ncCCDb0I52KGvaEyg1JgfKfs=s287-w287-h192-n-k-no-v1",
          "original_image": "https://www.iceportal.com/data/2813-48950-f75222340_XXLejpg/Germany/Berlin/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm/Photo/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm-Berlin-Undefined-Guest-room-bath.jpg"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/UwGUbYV0q-MG9raHnIRbA5TeTCCJgGsBhs-QUND765NOUTkB8xN_zv6WXrYveb2LcAHX1JfJAnJixEenYZisypkzVJi4A_hqBg1m6PaF4TNrphpGnD_v0YFApfjAKSw0lf9Rrs4bKBy0OZ-p68IEooey6rAlyg=s287-w287-h192-n-k-no-v1",
          "original_image": "https://www.iceportal.com/data/2813-48950-f75384466_XXLejpg/Germany/Berlin/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm/Photo/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm-Berlin-Undefined-Bathroom.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNj_KKQvBb7yND2IA91osPBwDmoBJd6dUjyiyVp=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNj_KKQvBb7yND2IA91osPBwDmoBJd6dUjyiyVp=s10000"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/YTQHX6puompQNtwT06MzzW4Bs86biq6RZVPdc1cQOzss11ZHWfQXajSFOfkxT9DJhQTl9TjZCdU-689LTcY_p_dSpXWim_ADWi_mpkjPkvCyzfa4PxiXGr72kzVRyQd94_GPYuqqQo0nvTyrJFgxhPLYA_un5Ds=s287-w287-h192-n-k-no-v1",
          "original_image": "https://www.iceportal.com/data/2813-48950-f75223416_XXLejpg/Germany/Berlin/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm/Photo/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm-Berlin-Undefined-Guest-room.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/ZwQV_JkcRu160_jipkUqPVoQad8dVJLBpkhTBS56DWMv6AaVh8CY1VYgjGSnLfhT71orzZ3OMCQwf2vRbPkllbXfO3HSUFqUtlPrL3vrzv-I0Jmsdlax5ylGNxtPwxGauMmfNv6aBzieMKdNIIhtXxRz9r9oQQE=s287-w287-h192-n-k-no-v1",
          "original_image": "https://www.iceportal.com/data/2813-48950-f75222308_XXLejpg/Germany/Berlin/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm/Photo/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm-Berlin-Undefined-Breakfast-area.jpg"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/OF3HX8wJqbbWKDF3fZymgs4IQcakhMnzvxkJoYn6L0Y3hgLGO6rHvltNYdpr8P7FkSE_K5Fb0ZZPGPSDx9LibUt-qP76tD7Ft6MLuEKQ3fgFAMhrwGCx-Ov2HpS3_DnTBmugHbVctp_ZCfB9RK8-CFL_Cd6_Hxs=s287-w287-h192-n-k-no-v1",
          "original_image": "https://www.iceportal.com/data/2813-48950-f75223400_XXLejpg/Germany/Berlin/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm/Photo/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm-Berlin-Undefined-Guest-room.jpg"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/mhe6JoMi8qvCwSPXr781SC3hA-ZR-c8z-1EC_r2754x5muXMDNHg3Z2UebXD0Ltardguark8yuw1eFbb4qOUviYfjTcn8DWEb6px7rr63haHIWSUMt7uIQs0Qn7_HFVbmCvcDFvvlw02NNdyMFLCGyHMrwBz1co=s287-w287-h192-n-k-no-v1",
          "original_image": "https://www.iceportal.com/data/2813-48950-f75417499_XXLejpg/Germany/Berlin/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm/Photo/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm-Berlin-Undefined-Premium-room.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPS8EhI1bUeaES-cbMLvdHwamkkMBvaKCuJv20G=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPS8EhI1bUeaES-cbMLvdHwamkkMBvaKCuJv20G=s10000"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/pw4JbZF7Jrr4XInYg1gspWAZgCpQQNTpO8MsWdqsQNaRTr8t7IlFdoLTFYvBH9KlQS5ZXcIpR4Ra39dCaoF9Mx2Rw11rwNEJKAX5s8yDQz_YLY996xWh_w4rPfA4_o1lpDxlm5rxtHeUy5dNPq8z-dAX0KSzygY=s287-w287-h192-n-k-no-v1",
          "original_image": "https://www.iceportal.com/data/2813-48950-f75222392_XXLejpg/Germany/Berlin/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm/Photo/BEST-WESTERN-Plus-Plaza-Berlin-Kurfuerstendamm-Berlin-Undefined-Breakfast-area.jpg"
        }
      ],
      "overall_rating": 4,
      "reviews": 1589,
      "ratings": [
        {
          "stars": 5,
          "count": 634
        },
        {
          "stars": 4,
          "count": 575
        },
        {
          "stars": 3,
          "count": 239
        },
        {
          "stars": 2,
          "count": 73
        },
        {
          "stars": 1,
          "count": 68
        }
      ],
      "location_rating": 4.8,
      "reviews_breakdown": [
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 245,
          "positive": 167,
          "negative": 48,
          "neutral": 30
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 343,
          "positive": 284,
          "negative": 25,
          "neutral": 34
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 389,
          "positive": 281,
          "negative": 74,
          "neutral": 34
        },
        {
          "name": "Restaurant",
          "description": "Restaurant",
          "total_mentioned": 108,
          "positive": 75,
          "negative": 20,
          "neutral": 13
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 271,
          "positive": 188,
          "negative": 62,
          "neutral": 21
        },
        {
          "name": "Air Conditioning",
          "description": "Air conditioning",
          "total_mentioned": 53,
          "positive": 12,
          "negative": 35,
          "neutral": 6
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Parking ($)",
        "Air conditioning",
        "Pet-friendly",
        "Bar",
        "Restaurant",
        "Room service",
        "Business centre",
        "Child-friendly",
        "Smoke-free property"
      ],
      "property_token": "ChgIh6fNppfg0OhFGgwvZy8xMmhzbGJ3YjkQAQ",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChgIh6fNppfg0OhFGgwvZy8xMmhzbGJ3YjkQAQ&q=Berlin+Hotels"
    },
    {
      "type": "vacation rental",
      "name": "Hotel-Pension Spree - Double Room",
      "link": "https://www.bluepillow.com/search?p_id=589ded2f7c00cb10c8d9d3da&dest=bkng&cat=House&accomodationids=594397b67c00cb0e643c52c8",
      "gps_coordinates": {
        "latitude": 52.49306106567383,
        "longitude": 13.329509735107422
      },
      "check_in_time": "2:00 PM",
      "check_out_time": "11:00 AM",
      "rate_per_night": {
        "lowest": "€67",
        "extracted_lowest": 67,
        "before_taxes_fees": "€60",
        "extracted_before_taxes_fees": 60
      },
      "total_rate": {
        "lowest": "€404",
        "extracted_lowest": 404,
        "before_taxes_fees": "€361",
        "extracted_before_taxes_fees": 361
      },
      "prices": [
        {
          "source": "Bluepillow.com",
          "logo": "https://www.gstatic.com/travel-hotels/branding/190ff319-d0fd-4c45-bfc8-bad6f5f395f2.png",
          "num_guests": 1,
          "rate_per_night": {
            "lowest": "€67",
            "extracted_lowest": 67,
            "before_taxes_fees": "€60",
            "extracted_before_taxes_fees": 60
          }
        }
      ],
      "nearby_places": [
        {
          "name": "Berlin Zoologischer Garten Bahnhof",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "7 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "24 min"
            },
            {
              "type": "Public transport",
              "duration": "45 min"
            }
          ]
        },
        {
          "name": "Taverna Hellas",
          "transportations": [
            {
              "type": "Walking",
              "duration": "5 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/lfTxW_YdolIDDlLCoT0TgZ9TE7DwF3UqLhyIhibh5V3RmdIPIN47InuvfA4VAmUuCVKZ5gPn3N74rZ5mjBfLxTNlzLgUyqTXl6IDYoCOEKKhmXFVsLBRiVcU4cZEHMHBkrEFC-6GJ1VkXtUkFdWA1j6GyZlPSjY=s287-w287-h192-n-k-no-v1",
          "original_image": "https://q-xx.bstatic.com/xdata/images/hotel/max1024x768/485417756.jpg?k=1a05015db666e47890d07733724573ba8b6893982040d57819f0dd3e4ee63c3d&o="
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/UasgzOj_Q6PMlwGMTGRMfbYWK_m4okncTXm8M6IixPuq2zw8_h1Aua15MmTTNj3VisAlR5Bfok05l_ESCBXgwCmPHZpfV3x0rSnwN4om7ovwaOnnIikXu3HMJC8NDFzkBRdRmASbW15EHq7jVi_4-aqnmUOTf9E=s287-w287-h192-n-k-no-v1",
          "original_image": "https://q-xx.bstatic.com/xdata/images/hotel/max1024x768/485414997.jpg?k=bd7e90d8c0ad42dc66fe8083a8b4375ec491e4110636bab93078b961af4f832d&o="
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/HwYopwgR66-O1VPMpMj9xF81gJhgPvZatZXa6eTp1Bbw5ctERjip49fF9FDaqkAggPjSeyzd0oJyRI0r84LMMxvKjuF5cx6yJu79hW5DXeghgF0ArqMpuIU3VfrcHC-duld4VQULXO29tIEhkqJKeqjg-yDqqms=s287-w287-h192-n-k-no-v1",
          "original_image": "https://q-xx.bstatic.com/xdata/images/hotel/max1024x768/196712650.jpg?k=a1ecc186077b217a0d59d551e2833f81dbfc70666505bf73160e15995cf07690&o="
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/NHTmsI8on4pEic0lVKFzMwuOQa-cWn3xBzRcS_HQB1HqVictH3dhzWPwaeVNoZkP5nJ4PZN7pejGmeqx_0HlMn-QhLIZDdlmCiQxABNy9b-eqAm_fka9F2h9tWwTxHxt4YVxmEzYyZyFJL-2l44u6Kiq5rEaAQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://q-xx.bstatic.com/xdata/images/hotel/max1024x768/229580507.jpg?k=ed3ff618e56cd4df28bd3d85181765d9c00e3788f94387f7ce67b76d26711704&o="
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/APLzYlWyQn3yjmDigQLSDoQAe2zfXeHtt9kZScnYyWN_gXTSubrjSjegnU9I4Tl2lkj7nSBVkbIiFes45Tm2P_FU5Wocx8hSpYTYz03xk1T8YGJZMJcVlWQn4ZdqPxeK1jH-mlt0DFzSEaR1oLtUE_HqNWeiSw=s287-w287-h192-n-k-no-v1",
          "original_image": "https://q-xx.bstatic.com/xdata/images/hotel/max1024x768/229581636.jpg?k=a5bd729a494d473d3ad64e011f5185e892a29fe4b8d8b108e7227d13b14c7231&o="
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/yzjtCJEjAamF1iJp81jkF_e4ZLWqlYUby6pPF4kXo69mimlc8s1UXYs37ONrmFfjdGEKdsFaznlto0F-sIxMYO0oOx42qDvTVQcpI61442OziidnVn2wrtE7Y0Y9eVrY9wsBhuX27wHZF8ZrNEypdbJB_ckPIg=s287-w287-h192-n-k-no-v1",
          "original_image": "https://q-xx.bstatic.com/xdata/images/hotel/max1024x768/229579399.jpg?k=2811ed0c1f5a6f27637bb9eafde20a22e99d6157394d8a853778eef3b973ad52&o="
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/lEgHDHMH76W3n2xpi4LUw5AoI_ESUOI_C8xCNA1fNHkzqV4_RMcKOCsW8evYLZhuwDudcMMMm8IdnXXS7zxvfZmisTfrpeXIDJPEMcvCXwewnb2HLvRN5th10eVecxK2LiClLot0LlWnun1Ybws2N0mpVglNJ6c=s287-w287-h192-n-k-no-v1",
          "original_image": "https://q-xx.bstatic.com/xdata/images/hotel/max1024x768/485415361.jpg?k=b0051099e42977b194e7613e2f77f979cf499701edeb30f290f1dd7963de326e&o="
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/Qwfzo-rnCCRQS3SirKATH9Lt8UOhdQfMOMrTSVMtoJTGgY1NLzrnjVDbVcqu0fsh3Zt3myFZIL6Os2CAaWir0PANqRzKTMaE6Zc-WzYHfckhrEoCEPmR13ln3jSembmGKGjSV6SX2rwi4NBytP1WZ10jMl78LA=s287-w287-h192-n-k-no-v1",
          "original_image": "https://q-xx.bstatic.com/xdata/images/hotel/max1024x768/485415042.jpg?k=ec72389c23619817483fe231ba9cab10e676ba08d22b1f43e65dd9237f2d6120&o="
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/NQHvuXSz1W5B7VLH4UxM53qMO_Kd3gj_QsDcUq_q8d92JPoCWcVu7QadyXDRIVJva3T4TTwfYk82siiwby6fJio_a1ELmlQxND-p300hF6hQ39nTFYsTztPBh7REhbVgP44LQI3uJA7zLsdT5BEwP_5V_6mwkw=s287-w287-h192-n-k-no-v1",
          "original_image": "https://q-xx.bstatic.com/xdata/images/hotel/max1024x768/485415686.jpg?k=e5553b5455252bcec45ed72d219514c893467fe67d95d8c8962c3a807517755b&o="
        }
      ],
      "overall_rating": 3.7,
      "reviews": 1407,
      "location_rating": 4.1,
      "amenities": [
        "Kid-friendly",
        "Crib",
        "Elevator",
        "Heating",
        "Kitchen",
        "Pet-friendly",
        "Smoke-free",
        "Cable TV",
        "Washer",
        "Wheelchair accessible",
        "Free Wi-Fi"
      ],
      "excluded_amenities": [
        "No air conditioning",
        "No airport shuttle",
        "No balcony",
        "No beach access",
        "No fireplace",
        "No fitness center",
        "No hot tub",
        "No indoor pool",
        "No ironing board",
        "No microwave",
        "No outdoor grill",
        "No outdoor pool",
        "No oven stove",
        "No patio",
        "No parking"
      ],
      "essential_info": [
        "Entire house",
        "Sleeps 2",
        "1 bedroom",
        "1 bathroom",
        "1 bed"
      ],
      "property_token": "ChoQmvnB6-D2iYKPARoNL2cvMTFzc2diNGZxNBAC",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChoQmvnB6-D2iYKPARoNL2cvMTFzc2diNGZxNBAC&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Mondrian Suites Hotel Berlin Checkpoint Charlie",
      "description": "Modern lodging offering streamlined suites with kitchen facilities & balconies or terraces.",
      "link": "http://www.mondriansuites.com/",
      "gps_coordinates": {
        "latitude": 52.5064586,
        "longitude": 13.3954021
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "11:00 AM",
      "rate_per_night": {
        "lowest": "€110",
        "extracted_lowest": 110,
        "before_taxes_fees": "€98",
        "extracted_before_taxes_fees": 98
      },
      "total_rate": {
        "lowest": "€660",
        "extracted_lowest": 660,
        "before_taxes_fees": "€589",
        "extracted_before_taxes_fees": 589
      },
      "deal": "37% less than usual",
      "deal_description": "Great Deal",
      "nearby_places": [
        {
          "name": "Checkpoint Charlie",
          "transportations": [
            {
              "type": "Walking",
              "duration": "7 min"
            }
          ]
        },
        {
          "name": "Bahnhof Berlin-Friedrichstraße",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "10 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "30 min"
            },
            {
              "type": "Public transport",
              "duration": "57 min"
            }
          ]
        },
        {
          "name": "Restaurant Choppaluna",
          "transportations": [
            {
              "type": "Walking",
              "duration": "4 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/_guChL4vX7um6q-mxP9BESIGlzqFAKwtT4yazIY38YTGfe-TtzbUhs_onRRZu_xbijsxG0SVbRYoWvb10JpYW4YN-Zb0K1msLkr0yNjOYRBKYBrvxcGIN52-pH860K5BAWb5YEdFpHghOxcxM1yFcaGHHea9sw=s287-w287-h192-n-k-no-v1",
          "original_image": "https://dynamic-media-cdn.tripadvisor.com/media/photo-o/04/85/08/87/mondrian-suites-hotel.jpg?w=1200&h=800&s=1"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/WFOAhq_UBzahAZDkE7nqLlY-OTrCv6cSo8lrkjvkSobz-6oxc9ArInpvDlkgGr4Rw9KFgT23fz6Pxr_3c-BlFhYSFoa4b1vUCPt-kp2fEOENfMbjfvgD8lFIxLjL5W-GdRblKfpi8tnZbJyqzW2O64unZYeeqw=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/6/0/88/768/260/Comfort_en_suite_S.jpg"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/nmIm89xvsF2lNIxoMn98H-h1KFA7j9aZmIqcK2LxpZ7ou24GB0jLpl40nI28v5lP4Sh1sp-xOT7Yk1LVj-eG9XTiPp0z9HLi3--1rnZgftIpjPW6-z84wcGjyLMUWzSpLwH43AK0kjBxmeXNLmFnWd96sWIuAB8=s287-w287-h192-n-k-no-v1",
          "original_image": "https://images.trvl-media.com/lodging/24000000/23980000/23976600/23976534/572c0858_z.jpg"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/mowcObTp5kEpFR3za0EO7fmEJQAJfz9ywj9No4FCq1IQmsrqzgvpIvIAvslxoq79qzaExG8Uh4Q4g1a5WXYl7vB11WoLSO6Kl2sK8uKmCfB88v1sK5mJ8LBmeqVSdprG23O65PB-Nn5UP1kpKhxrsf9DtBNGaQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://dynamic-media-cdn.tripadvisor.com/media/photo-o/04/85/08/70/mondrian-suites-hotel.jpg?w=1200&h=800&s=1"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/3-Z1gzGs69XqZDbrt6DzlW8yEYnhZctiwnmrM1_mw4U6UJN7lcxUiGz3ac_PIeKIcm7MAm3jDlrZNwrMGt1U1nk-H04g_a5Y2gaqCuKuLSbhpmB0tCuMi8mKon32M8Q9-UlKZlLUcIOkzMD9RPcp3GvvYpGOCH0=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/6/0/88/768/340/Suite_2_S.jpg"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/BYfu2aN2ZUKlzz0xVqwLu_8xvL_vUD7q_TPj9JVlsVwNtHnL9NPiaP5wZE8H6NIb3tRhxJKOcQ5Y5DUht_2TvwI6phyzxRLeAyBoQgE5oSLhOCeYei7rqsxnn1ycr2glLkhEqzzA6NOo00DMn--vEYE9OcaUig=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/6/0/88/768/276/Nespresso_S.jpg"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/Ejnz9kjdLI4WJakgBvqzLJ-mTnIC9pbpXV2Dg-XiDbE0IMaRB_X7FRKTtj65xPDeGjMUu5-wvPzMo_6-_Pb1S3SM6xZeMEWZQIXbE_NwkW6KDcg8Zws2bv1Z_iEbNr0unxOtFtZ9DixT6WmMNQRwIJAhqXmMX3k=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/2/0/51/456/151/RESTAURANT_S.jpg"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/stMBHI0UjkHugrJW07OnULIrPYPw63W3R4EHr4W6LZ-Edctv_xduU9583wT6Sj5--0zcio2mJMMpjbFagcekKFH5G0oey9VlYdkmR1UqU_7ZnDoWUSGpfJGc6ZQM1Zu_5E3vvFT2lALEr9bsCEJpHBQBzntg2Jk=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/6/0/88/768/252/Apartment_Deluxe_S.jpg"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/3Kffek7NLzdXOwcjDX5Zo4wnIkmdoD15vPdygF_xH9DQrxJUTcGQ3-ABm4ue-MPCoWFMpTzRe1crUAi8Tu8v1Bz_JLDXjAQaxN95hNWDJnmgr8XFhG7e8b3rPKTDGpRw5D22xwlXaG3PQh4czUKpvZ0-mvjY5A=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/6/0/88/768/348/Suite_3_S.jpg"
        }
      ],
      "overall_rating": 4.3,
      "reviews": 1109,
      "ratings": [
        {
          "stars": 5,
          "count": 607
        },
        {
          "stars": 4,
          "count": 323
        },
        {
          "stars": 3,
          "count": 94
        },
        {
          "stars": 2,
          "count": 36
        },
        {
          "stars": 1,
          "count": 49
        }
      ],
      "location_rating": 4.6,
      "reviews_breakdown": [
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 251,
          "positive": 226,
          "negative": 11,
          "neutral": 14
        },
        {
          "name": "Kitchen",
          "description": "Kitchen",
          "total_mentioned": 93,
          "positive": 65,
          "negative": 16,
          "neutral": 12
        },
        {
          "name": "Room",
          "description": "Room amenities",
          "total_mentioned": 119,
          "positive": 56,
          "negative": 43,
          "neutral": 20
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 284,
          "positive": 216,
          "negative": 39,
          "neutral": 29
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 198,
          "positive": 148,
          "negative": 48,
          "neutral": 2
        },
        {
          "name": "Wellness",
          "description": "Wellness",
          "total_mentioned": 15,
          "positive": 11,
          "negative": 1,
          "neutral": 3
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Parking",
        "Pet-friendly",
        "Restaurant",
        "Kitchen in rooms",
        "Child-friendly"
      ],
      "property_token": "ChkIzdaRzZvuxbWSARoML2cvMTJobWJfbjgxEAE",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChkIzdaRzZvuxbWSARoML2cvMTJobWJfbjgxEAE&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Hotel Riu Plaza Berlin",
      "description": "Contemporary rooms in a city center property offering a sleek bar & a buffet restaurant.",
      "link": "https://www.riu.com/en/hotel/germany/berlin/hotel-riu-plaza-berlin/?utm_source=google&utm_medium=organic&utm_campaign=my_business&utm_content=ZBE",
      "gps_coordinates": {
        "latitude": 52.5003447,
        "longitude": 13.3468144
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "€113",
        "extracted_lowest": 113,
        "before_taxes_fees": "€108",
        "extracted_before_taxes_fees": 108
      },
      "total_rate": {
        "lowest": "€678",
        "extracted_lowest": 678,
        "before_taxes_fees": "€646",
        "extracted_before_taxes_fees": 646
      },
      "nearby_places": [
        {
          "name": "Reichstag Building",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "8 min"
            }
          ]
        },
        {
          "name": "Berlin Zoologischer Garten Bahnhof",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "7 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "26 min"
            },
            {
              "type": "Public transport",
              "duration": "53 min"
            }
          ]
        },
        {
          "name": "Golden Rice Kudamm",
          "transportations": [
            {
              "type": "Walking",
              "duration": "3 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOJr_osUkEeFOzrc3bhkW6Tyg76QMRZui7RyIHr=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOJr_osUkEeFOzrc3bhkW6Tyg76QMRZui7RyIHr=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipN1Kwc8QjWGDDyt8KUI6aqJ9j_6XjVywWn-HE41=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipN1Kwc8QjWGDDyt8KUI6aqJ9j_6XjVywWn-HE41=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOW3trboJleAKpdt1E148ADY3sqtZkTlDF4XXfY=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOW3trboJleAKpdt1E148ADY3sqtZkTlDF4XXfY=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPSoG9ecDxlhXuQtrlf5qJ7wuQHADiDiRI0ibpD=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPSoG9ecDxlhXuQtrlf5qJ7wuQHADiDiRI0ibpD=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNDrGtpKmJvrH57LBUsHjQxe6pf39zLzqbA6fLQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNDrGtpKmJvrH57LBUsHjQxe6pf39zLzqbA6fLQ=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOwPQE3ITNHxIUJxcIh1lEOgQZ-48txTBBhwHOv=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOwPQE3ITNHxIUJxcIh1lEOgQZ-48txTBBhwHOv=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNlQgWZ1IhZFdDCxUTVtZDQbWzO27_ZtYWK9p74=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNlQgWZ1IhZFdDCxUTVtZDQbWzO27_ZtYWK9p74=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipNt6fyzkU8ajQFuQQW-RWr0XMIHF7rHejZKOEFH=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipNt6fyzkU8ajQFuQQW-RWr0XMIHF7rHejZKOEFH=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPmz_0RKiB8I45sttsqFdFd9nqeu9ZU8mdPLIW5=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPmz_0RKiB8I45sttsqFdFd9nqeu9ZU8mdPLIW5=s10000"
        }
      ],
      "overall_rating": 4.4,
      "reviews": 5246,
      "ratings": [
        {
          "stars": 5,
          "count": 3218
        },
        {
          "stars": 4,
          "count": 1413
        },
        {
          "stars": 3,
          "count": 346
        },
        {
          "stars": 2,
          "count": 112
        },
        {
          "stars": 1,
          "count": 157
        }
      ],
      "location_rating": 4.9,
      "reviews_breakdown": [
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 961,
          "positive": 795,
          "negative": 100,
          "neutral": 66
        },
        {
          "name": "Parking",
          "description": "Parking",
          "total_mentioned": 414,
          "positive": 279,
          "negative": 84,
          "neutral": 51
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 1020,
          "positive": 918,
          "negative": 23,
          "neutral": 79
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 1480,
          "positive": 1249,
          "negative": 137,
          "neutral": 94
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 1082,
          "positive": 881,
          "negative": 155,
          "neutral": 46
        },
        {
          "name": "Restaurant",
          "description": "Restaurant",
          "total_mentioned": 414,
          "positive": 356,
          "negative": 35,
          "neutral": 23
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Free parking",
        "Air conditioning",
        "Fitness centre",
        "Bar",
        "Restaurant",
        "Full-service laundry",
        "Accessible"
      ],
      "property_token": "ChkI3qKu7KzvtMnxARoML2cvMTJxZzQ2MXNwEAE",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChkI3qKu7KzvtMnxARoML2cvMTJxZzQ2MXNwEAE&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Hotel Motel One Berlin-Alexanderplatz",
      "description": "Fashionable hotel featuring an industrial-chic bar & lounge, plus complimentary Wi-Fi.",
      "link": "https://www.motel-one.com/hotels/berlin/hotel-berlin-alexanderplatz",
      "gps_coordinates": {
        "latitude": 52.519571899999995,
        "longitude": 13.4124216
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "€118",
        "extracted_lowest": 118,
        "before_taxes_fees": "€105",
        "extracted_before_taxes_fees": 105
      },
      "total_rate": {
        "lowest": "€708",
        "extracted_lowest": 708,
        "before_taxes_fees": "€630",
        "extracted_before_taxes_fees": 630
      },
      "nearby_places": [
        {
          "name": "Alexanderplatz",
          "transportations": [
            {
              "type": "Walking",
              "duration": "3 min"
            }
          ]
        },
        {
          "name": "Alexanderplatz",
          "transportations": [
            {
              "type": "Walking",
              "duration": "5 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Public transport",
              "duration": "34 min"
            },
            {
              "type": "Taxi",
              "duration": "34 min"
            }
          ]
        },
        {
          "name": "Restaurants Vincenzo",
          "transportations": [
            {
              "type": "Walking",
              "duration": "2 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/Zcupqkh4pd_DMPuat5YjDxL5av3MFVSMo-QxZagTWvIDj_O7rpdCL6laDBz6NmoRd12ZjyK6l03ZgKf5Nm-TciS6TUnPGusxPKgmuLsLsMNEmLnyc6prKcSdGpjT8VhatpfLNmqw_W0DeZ1OLHEqpY5wLK8tI0M=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/107/816/163/MO_Berlin-Alexanderplatz_Room_13_O.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipPDHn6EaP7wZdSpkuMkWK6-z7Qu6sTUfg5aVtq8=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipPDHn6EaP7wZdSpkuMkWK6-z7Qu6sTUfg5aVtq8=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/rIQ5QCsckW-rLzFp4sF02KfcDJvUkPCxYfUcO_RR2_pJ8aVQJNtfdQM71PXk6AoPy8cL_egKPfumUEFv7X7PtFgELeHn9oXXY0IqkS0CPKNAneCMxpFsCHb6iVgvcF4udqmgNvLE1bo0qhcgpuGZEkre864xDXk=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/107/816/171/MO_Berlin-Alexanderplatz_Room_16_O.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/5uo-k1MQxBjmYvPscLOVnRXeCO4oYqVBJX5ivqFsZTV-6yCXmUka1d310kpC7lE0STaykUuKIgbR9YCFcH09D12LBF1HQP17IJItbXwm0lObxJ1hP_Ga_-K1Y1Cg5WpVo5bGoeGVE8FDgYVEXMXq5WpsZ-NzrDQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/107/816/267/MO-Fr_hst_ck-09_S.jpg"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/AjoFvF7hephZD8M8u2EtdTiOhXZ2yui3BukeO_VuASovBUlkBOsrHI08CxyX-Lxc-9lQzKTjONo_stCelNuM2_NivsQPFrawnJzMKxQkkg-oSs5CCkSmgigbGtkBedtSIgLqfkYR55aPenKHgMlQ9jjOV2F1kAA=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/166/681/29/150215474_Motel_One_Berlin-Alexanderplatz_Breakfast_Buffe_O.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipOmP_37R7G56dkaBE3S8kBTO8zsBb3MKx_s7F63=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipOmP_37R7G56dkaBE3S8kBTO8zsBb3MKx_s7F63=s10000"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/cG0eX9CleaLXhfOva7Ja-wNP2gHP2z5vl9NB1_ews3cViSfSvi43g8uAmH_RWXwTLNCNPyjEFABUP5mNswcKEpUZ5BFF1CjflwmmlB_05hBClfAKTZIJTCJ5D0Q4My03f8gkl-VtXkS2lSgU9Fuql0jzEqSJAw=s287-w287-h192-n-k-no-v1",
          "original_image": "https://d2hyz2bfif3cr8.cloudfront.net/imageRepo/7/0/107/816/243/MO-Fr_hst_ck-06_S.jpg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipP1PoFKMU6XF5kfB7XvIqX64w8492QgUCbE8vA_=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipP1PoFKMU6XF5kfB7XvIqX64w8492QgUCbE8vA_=s10000"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/p/AF1QipP9m069aizJYE0FecBOpfhbyZ0m3PDJHK3uq2ob=s287-w287-h192-n-k-no-v1",
          "original_image": "https://lh5.googleusercontent.com/p/AF1QipP9m069aizJYE0FecBOpfhbyZ0m3PDJHK3uq2ob=s10000"
        }
      ],
      "overall_rating": 4.5,
      "reviews": 3534,
      "ratings": [
        {
          "stars": 5,
          "count": 2376
        },
        {
          "stars": 4,
          "count": 895
        },
        {
          "stars": 3,
          "count": 149
        },
        {
          "stars": 2,
          "count": 51
        },
        {
          "stars": 1,
          "count": 63
        }
      ],
      "location_rating": 5.0,
      "reviews_breakdown": [
        {
          "name": "Bar",
          "description": "Bar or lounge",
          "total_mentioned": 398,
          "positive": 331,
          "negative": 43,
          "neutral": 24
        },
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 499,
          "positive": 325,
          "negative": 107,
          "neutral": 67
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 741,
          "positive": 677,
          "negative": 23,
          "neutral": 41
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 1005,
          "positive": 828,
          "negative": 96,
          "neutral": 81
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 651,
          "positive": 570,
          "negative": 67,
          "neutral": 14
        },
        {
          "name": "Restaurant",
          "description": "Restaurant",
          "total_mentioned": 216,
          "positive": 134,
          "negative": 51,
          "neutral": 31
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Air conditioning",
        "Pet-friendly",
        "Bar",
        "Restaurant",
        "Accessible",
        "Child-friendly",
        "Smoke-free property"
      ],
      "eco_certified": True,
      "property_token": "ChkIs57a75i1-Z8pGg0vZy8xMWYwM21senczEAE",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChkIs57a75i1-Z8pGg0vZy8xMWYwM21senczEAE&q=Berlin+Hotels"
    },
    {
      "type": "hotel",
      "name": "Vienna House Easy by Wyndham Berlin Potsdamer Platz",
      "link": "https://www.wyndhamhotels.com/vienna-house/berlin-germany/vienna-house-easy-berlin-potsdamer-platz/overview?CID:LC:6qt7c54dekbf1g7:58292",
      "gps_coordinates": {
        "latitude": 52.499831099999994,
        "longitude": 13.3710389
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "12:00 PM",
      "rate_per_night": {
        "lowest": "€88",
        "extracted_lowest": 88,
        "before_taxes_fees": "€79",
        "extracted_before_taxes_fees": 79
      },
      "total_rate": {
        "lowest": "€529",
        "extracted_lowest": 529,
        "before_taxes_fees": "€472",
        "extracted_before_taxes_fees": 472
      },
      "nearby_places": [
        {
          "name": "Brandenburg Gate",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "9 min"
            }
          ]
        },
        {
          "name": "Berlin Central Train Station",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "10 min"
            }
          ]
        },
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "29 min"
            },
            {
              "type": "Public transport",
              "duration": "55 min"
            }
          ]
        },
        {
          "name": "Grimm's Tischlein deck Dich",
          "transportations": [
            {
              "type": "Walking",
              "duration": "5 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/yRAcnRhCH9hM5epCR_RrbaqGu1yvZ42SBja3cfwpIgGmaO7EjKamyWh45SQKSH7wmQXaTvUwYZIHU5G_9_ypxlL56rfvDth5j00gbEdEyLQWWH7CAmwBkjVLraBu0vJkekbPuQF96UmYVdH1641Gv-HK1Q0D1A=s287-w287-h192-n-k-no-v1",
          "original_image": "https://i.giatamedia.com/m.php?m=AQABAAAAla4KAB2ypwYFALO9ceTTspbYYL1YWcciirE"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/twioGtNCpxpDC01Prj1duOVRAL3y9IZke3TywQZ7K4r7rDMTXwDGfS8F4nDoyOYjnmdTPvurjO7tsLMbbINBUIBfQzhlUX44UhrAh-V5-iaRC5EeZZg1qT8vwFVTY17tG5a6Ag-CP1UJbJX_zbGjLIUlDAD2jg=s287-w287-h192-n-k-no-v1",
          "original_image": "https://i.giatamedia.com/m.php?m=AQABAAAAla4KAJLXnwYFADsyKRDoHXPWO9aos0GgFRg"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/fRub0xYCsNrxAMpql0OJlyzUsvuIhHLBmPb1l7UZILmUveyirDk5GCee_vldQ-Zz7bwcNa5vU9XN7FEC19ec0mVdL9Co1zBtF3XLN_TK3HOtY6yzHvDPy6beAFuxwj8WtJ5awKAhxk9ypjxZvL9DGs0J2NqoBBs=s287-w287-h192-n-k-no-v1",
          "original_image": "https://i.giatamedia.com/m.php?m=AQABAAAAla4KAA2ypwYFABSrHzMO2fy8oPeoAZKxSRQ"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/ofg8NPef-wPOrSYVVK7o5NPdj9tI5owSGe8PSN2vnjmdXc_kG6RMl4StbP9I0qoGj6RydLDW5uGZHWjSUO88xvJAzsu4jfleievm6CROsyEJIR6GUZoDiJC92VMLnLpSWPZKZqXql7aikiEKY-ikijkqHhly1g=s287-w287-h192-n-k-no-v1",
          "original_image": "https://cdn.worldota.net/t/1024x768/content/bf/66/bf66c71b626d6f0966a3e40e20bf46f5f598ce10.jpeg"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/PupcwpHSVUffND0h78SAaUL3M7qU9IDIfgmdbNNjQON-tgwJW8QLm8BzbQQFMoqISn5lTi1cgJEE_UX2hsm4Fn4aelmXqasvTfcVp2m_6h6UiMnwo-57nDnSbye34SKBaSuFxwHctNqXh2mFhO2NnDydVtzggQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://i.giatamedia.com/m.php?m=AQABAAAAla4KAIzXnwYFAA0QLV-Mt0HOc3d85mjqzhY"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/c9W8MN-GyVIU7UP-i1hg-vKxMLLluFu9S-_wiKbbV6QWCngQWX-o5mNKbxk-cQlfaBQNPNw24T7AtadNUp_7dLzuoBYkJjJDijg_uWIoQ8HWnxh1vc99JoT8nwjKR-mE69tpCnDMusJf2L8y5BqvPKS7_QZ3x5k=s287-w287-h192-n-k-no-v1",
          "original_image": "https://i.giatamedia.com/m.php?m=AQABAAAAla4KAAeypwYFAIX5o6BC8xtmUq6i4eONnvE"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/564AFbpbkqhuyVaEtoVr85zAfzVq2fOPrYRObZttwn-VVOgZabYQHhC_JVZeMLqWyem9XztgaUa02FO_20_51QTGhfVUKUeMEGcqaAgF3bwNdHyadpaZJZrR2Xm5w52pH9tg6p3y68KIAMuBz18lxWub1YZ5UAs=s287-w287-h192-n-k-no-v1",
          "original_image": "https://i.giatamedia.com/m.php?m=AQABAAAAla4KAJPXnwYFABwmYOeA-DQYOW3OeSeluX0"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/WeAJeKdV6bxt-4c8ztRYnVnBq4hDgCS8GTaf5fexl2UK73cPrnbkquqgEbaiNDjZpBoEIq9AQVFmqGQlhiKwQsjjXx4l5_yHUjCaj-teb0-NzG-3P2QTixreLpFwERJ4kZelYCB4QF2lg6b4-HzjKYZnKWD3FoM=s287-w287-h192-n-k-no-v1",
          "original_image": "https://i.giatamedia.com/m.php?m=AQABAAAAla4KABqypwYFANpA11YSmLB-88IErJC7ubI"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/B95FF3646HlGuz3zCfym7fo-BK92D6v_L5aPpgowDah6RFTp-5ziUhaeMM6ndMINZklO6lI0SmcQqZ9dwmHFzZX72abYFMpkA8SUVnGxxpNbMgd3nJIMMNuLBE5Y6sskGRdUPmq8cyiz9nk0cLaEZjzoV6YuiQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2b/43/1a/ee/caption.jpg?w=1900&h=1400&s=1"
        }
      ],
      "overall_rating": 4.3,
      "reviews": 168,
      "ratings": [
        {
          "stars": 5,
          "count": 108
        },
        {
          "stars": 4,
          "count": 37
        },
        {
          "stars": 3,
          "count": 5
        },
        {
          "stars": 2,
          "count": 4
        },
        {
          "stars": 1,
          "count": 14
        }
      ],
      "location_rating": 3.9,
      "reviews_breakdown": [
        {
          "name": "Breakfast",
          "description": "Breakfast",
          "total_mentioned": 59,
          "positive": 56,
          "negative": 2,
          "neutral": 1
        },
        {
          "name": "Service",
          "description": "Service",
          "total_mentioned": 62,
          "positive": 46,
          "negative": 14,
          "neutral": 2
        },
        {
          "name": "Location",
          "description": "Location",
          "total_mentioned": 53,
          "positive": 43,
          "negative": 4,
          "neutral": 6
        },
        {
          "name": "Sleep",
          "description": "Sleep",
          "total_mentioned": 33,
          "positive": 15,
          "negative": 14,
          "neutral": 4
        },
        {
          "name": "Restaurant",
          "description": "Restaurant",
          "total_mentioned": 24,
          "positive": 23,
          "negative": 0,
          "neutral": 1
        },
        {
          "name": "Property",
          "description": "Property",
          "total_mentioned": 64,
          "positive": 51,
          "negative": 9,
          "neutral": 4
        }
      ],
      "amenities": [
        "Breakfast ($)",
        "Free Wi-Fi",
        "Free parking",
        "Restaurant"
      ],
      "property_token": "ChoImaHEwpCdiZjsARoNL2cvMTF2NjZ2M3prahAB",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChoImaHEwpCdiZjsARoNL2cvMTF2NjZ2M3prahAB&q=Berlin+Hotels"
    },
    {
      "type": "vacation rental",
      "name": "Arbio I 1A Family room near Schonhauser Allee",
      "link": "https://book.arbio.com/listings/116778?googleVR=1",
      "gps_coordinates": {
        "latitude": 52.54520034790039,
        "longitude": 13.411600112915039
      },
      "check_in_time": "3:00 PM",
      "check_out_time": "10:00 AM",
      "rate_per_night": {
        "lowest": "€66",
        "extracted_lowest": 66,
        "before_taxes_fees": "€57",
        "extracted_before_taxes_fees": 57
      },
      "total_rate": {
        "lowest": "€397",
        "extracted_lowest": 397,
        "before_taxes_fees": "€345",
        "extracted_before_taxes_fees": 345
      },
      "prices": [
        {
          "source": "Arbio Group ",
          "logo": "https://www.gstatic.com/travel-hotels/branding/icon_default.png",
          "official": True,
          "num_guests": 1,
          "rate_per_night": {
            "lowest": "€66",
            "extracted_lowest": 66,
            "before_taxes_fees": "€57",
            "extracted_before_taxes_fees": 57
          }
        }
      ],
      "nearby_places": [
        {
          "name": "Berlin Brandenburg Airport",
          "transportations": [
            {
              "type": "Taxi",
              "duration": "50 min"
            },
            {
              "type": "Public transport",
              "duration": "50 min"
            }
          ]
        },
        {
          "name": "Villa Rodizio",
          "transportations": [
            {
              "type": "Walking",
              "duration": "5 min"
            }
          ]
        }
      ],
      "images": [
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/ypYLRpVIcBr8Gc_Dk_AQ7g99_CgENk0B-XGd25WjIMOUIA9zeWwt6QRmvYKtZi6S2mGkCLn-cLgpEyDY7fA43OaQneIdyrM-YhVuG1ovvUKLdUbY9PAHUZIFwRP8yBVMMDVRNaSlVtzurA28-XyNtEMmjxtB6LY=s287-w287-h192-n-k-no-v1",
          "original_image": "https://hostaway-platform.s3.us-west-2.amazonaws.com/listing/41334-116778-vt9wBoPYxHyrxQV--D8V2Ll46CQnqV-QyZkoK7Y7Ge7c-62d575003bc7a-medium"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/6-LFK1y9QM1kHeefelHzIuX0Vwy6MGGqcj7II_FCNWMn-vusBnhZnwDQ3qOTy2Y8eG6wPTK5s0sNrB6Parp0_fWzwBCPzrI50i--NHpxzu5xMSmBQFaddx0gwSs7Xuev5z-dCbC4HAywg73t35-lsWDdYUJm7Mg=s287-w287-h192-n-k-no-v1",
          "original_image": "https://hostaway-platform.s3.us-west-2.amazonaws.com/listing/41334-116778-nQef7MkbRNxAmNfCqrYJIkb5KfPo8o6iEoEVIWtCFo0-642d92b21a0bb-medium"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/AdcOJos9wOatTP6gDUPqcR3IuMoIKFLkpAG0ipxsfRJvl7I9s5_0RRcisU306yDAJWw-UZv8YY6sUJBP9nsEbM2ZT1SD84qmDOT3QCsJ16y12bH_DjZjRDRJB1N_8bMGl16lI_-MSPMMCW1ajxA9cgqJfPLRgQ=s287-w287-h192-n-k-no-v1",
          "original_image": "https://hostaway-platform.s3.us-west-2.amazonaws.com/listing/41334-116778-fDt6afzeigVhN--FUDPrKDTocOannXO61AF4Ft93BJKM-642d92b4bf06e-medium"
        },
        {
          "thumbnail": "https://lh6.googleusercontent.com/proxy/WeWJoFg-yiMeXzdk6kKt-0EUjffCjF69MyZHjqYHJaqYXchotfHYm9CwmIyiXyU4Nc16zR7RD0engM5Dcf4QasIv8Uu0VAE5pICs2cdyxDbT7ToT-M1VRcIENqUIZ4UnPyRMWTl-1BAH5Ms7mrC-OStOZIi4CA=s287-w287-h192-n-k-no-v1",
          "original_image": "https://images.trvl-media.com/lodging/94000000/93410000/93408500/93408422/8c5fe242.jpg?impolicy=fcrop&w=1000&h=666&quality=medium"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/jaPTOs7AItq96mjxXgqwns43sbRN_oinYxjR8pFMZi0o0k29Y9UYaC7go2q9KKLLKPgI9hxBwT86loSX5lpp8A_Zij4bjMoJWR5ur645H1O8T6Xuo83ashg_Pa-FHjvn7wx1P1hdAihsg3PsGHHo0w-yYDuL3yM=s287-w287-h192-n-k-no-v1",
          "original_image": "https://images.trvl-media.com/lodging/94000000/93410000/93408500/93408422/069b6f82.jpg?impolicy=fcrop&w=1000&h=666&quality=medium"
        },
        {
          "thumbnail": "https://lh5.googleusercontent.com/proxy/fGC-VHJjiqdVxEY6aJtZ5XR6WpRfxKtN_U5peLRGrhWr5-4qIk-dbIpUqeBxEfILaFX8V6HKxmPkDkObhFJ-0niWw8dGMeriKSrxkmhc9Ka5yQemi_k3sgMR8wlLpKM-Orri95H2rYFAWnXSuia_fHQ2JLPeVA=s287-w287-h192-n-k-no-v1",
          "original_image": "https://images.trvl-media.com/lodging/94000000/93410000/93408500/93408422/cdc4de2d.jpg?impolicy=fcrop&w=1000&h=666&quality=medium"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/vLXjV_VpGLEhbwVT5yH-whD7-ZgYRu89LkM7MCuQ6Hakr_KwWL1CCAe4XgcfUucq4hzUUQqjN4mqhui6dDsw2ncgtdz4fI0KzHBAQQfH84ddFoDNp05iqOAJP6ElurHyuymr0TZFZ76LwidrZ1prMfdyj1svalE=s287-w287-h192-n-k-no-v1",
          "original_image": "https://hostaway-platform.s3.us-west-2.amazonaws.com/listing/41334-116778-84Xyufa0RpjIa6Rztf-pr28HSe58L4888ypDIuUAkrE-642d92ba43a9d-medium"
        },
        {
          "thumbnail": "https://lh4.googleusercontent.com/proxy/te0yQQwbb8wnDuemCDcDSUiVrWeJhPbboqiMvF7GU3qKLjkzwiX51GOR0b6eMN5ZtXeQFKf6xpemfhyCl48tdPHR9mfu5rmZuc2qwFWjVfoOTknNkSex4i7qkFix1IloSE88AEbEDAeLjAWkEgyCtje5M1lAZA=s287-w287-h192-n-k-no-v1",
          "original_image": "https://hostaway-platform.s3.us-west-2.amazonaws.com/listing/41334-116778-qgSLNIicsUAnuzKgp3pycylngBYMtUviverqN3qRN48-642d92bf33f46-medium"
        },
        {
          "thumbnail": "https://lh3.googleusercontent.com/proxy/z44b5YyFWeYtsHdqK_fdDTz_6ZG_QzipFMNYJlWrAqbvGMLKKq5eXTtmLMaX1UcvAbBD376rFvJ_0A1rR4tun-rOs9UbO7ZAaFAEFEC7DlHlO3zu92I0SGqMD3snK_uzx1KTV28MnToJzTrxOYpqWRs0vWc59w=s287-w287-h192-n-k-no-v1",
          "original_image": "https://images.trvl-media.com/lodging/94000000/93410000/93408500/93408422/3d58997f.jpg?impolicy=fcrop&w=1000&h=666&quality=medium"
        }
      ],
      "overall_rating": 3.75,
      "reviews": 81,
      "location_rating": 4.4,
      "amenities": [
        "Heating",
        "Smoke-free",
        "Cable TV"
      ],
      "excluded_amenities": [
        "No air conditioning",
        "No airport shuttle",
        "No balcony",
        "No beach access",
        "Not kid-friendly",
        "No crib",
        "No elevator",
        "No fireplace",
        "No free breakfast",
        "No hot tub",
        "No indoor pool",
        "No ironing board",
        "No kitchen",
        "No microwave",
        "No outdoor grill",
        "No outdoor pool",
        "No oven stove",
        "No patio",
        "Not pet-friendly",
        "No washer",
        "Not wheelchair accessible",
        "No parking",
        "No Wi-Fi"
      ],
      "essential_info": [
        "Entire apartment",
        "Sleeps 3",
        "1 bedroom",
        "1 bathroom",
        "2 beds",
        "278 sq ft"
      ],
      "property_token": "ChkQ5oe0tI3Zs4ZxGg0vZy8xMXc0MnRuZnFrEAI",
      "serpapi_property_details_link": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&property_token=ChkQ5oe0tI3Zs4ZxGg0vZy8xMXc0MnRuZnFrEAI&q=Berlin+Hotels"
    }
  ],
  "serpapi_pagination": {
    "current_from": 1,
    "current_to": 20,
    "next_page_token": "CBI=",
    "next": "https://serpapi.com/search.json?adults=1&check_in_date=2024-08-30&check_out_date=2024-09-05&children=0&currency=EUR&engine=google_hotels&gl=us&hl=en&next_page_token=CBI%3D&q=Berlin+Hotels"
  }
}

In [17]:
def extract_hotels(api_response_json):
    return [{
        'hotel_id': x,
        'type': element.get('type'),
        'name': element.get('name'),
        'description': element.get('description',""),
   #     'latitude': element['gps_coordinates']['latitude'],
   #     'longitude': element['gps_coordinates']['longitude'],
        'number_reviews': element.get('reviews', 0),
        'total_cost': element['total_rate']['extracted_lowest'],
        'location_rating': element.get('location_rating'),
        'overall_rating': element.get('overall_rating'),
        'nearby_places': element.get('nearby_places'),
        'reviews_breakdown': element.get('reviews_breakdown', []),
        'amenities': element['amenities'],} for x, element in enumerate(api_response_json['properties'])]


In [18]:
extracted_hotels = extract_hotels(HOTEL_API_RESPONSE)
extracted_hotels
#pd.DataFrame(extracted_hotels).sort_values('total_cost').head(10)

[{'hotel_id': 0,
  'type': 'hotel',
  'name': 'MEININGER Hotel Berlin Mitte Humboldthaus',
  'description': 'Hip lodging with private rooms & mixed dorms, plus a lounge/snack bar, a guest kitchen & free Wi-Fi.',
  'number_reviews': 2914,
  'total_cost': 169,
  'location_rating': 5.0,
  'overall_rating': 4,
  'nearby_places': [{'name': 'Bode-Museum',
    'transportations': [{'type': 'Walking', 'duration': '7 min'}]},
   {'name': 'Berlin Central Train Station',
    'transportations': [{'type': 'Taxi', 'duration': '8 min'}]},
   {'name': 'Berlin Brandenburg Airport',
    'transportations': [{'type': 'Taxi', 'duration': '39 min'},
     {'type': 'Public transport', 'duration': '45 min'}]},
   {'name': 'Kamala',
    'transportations': [{'type': 'Walking', 'duration': '1 min'}]}],
  'reviews_breakdown': [{'name': 'Location',
    'description': 'Location',
    'total_mentioned': 729,
    'positive': 644,
    'negative': 28,
    'neutral': 57},
   {'name': 'Sleep',
    'description': 'Sleep',
 

In [19]:
prompt_generation = """Below it is provided a proxy (in JSON format) of the travel plan that the user wants to perform:
{travel_plan}
\nIn the JSON below I provide the best hotels available in {destination_place}
{hotels_json}
\nPlease, based on this travel plan, select the best accomodation for the user.
Here are the user preferences for the travel plan: '{user_preferences}'.
The general parameters to take into account for the best hotel should be (if the user has provided her/his preferences, you should assign to the following parameters a secondary priority):
- accomodation price
- number of reviews and ratings from other travellers
- distance from the attractions suggested in the travel plan
- services/amenities offered by the accomodation
The output must match the following JSON format: 
{{"hotel_id": "chosen_accomodation_id", "reason": "reason for this accomodation choice"}}
"""

travel_plan_prompt_generator = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=("You are an helpful travel plan assistant that generates travel plans based on user's requests"
                          "It is important that the assistant respects the requested output formats."
                          "You must only answer with just the answer to the request, avoiding any kind of conversational sentence.")),
        HumanMessagePromptTemplate.from_template(prompt_generation),
    ]
)

openai_agent = ChatOpenAI(**PARAMETERS['gpt_params'])

update_prompt = travel_plan_prompt_generator#| openai_agent | StrOutputParser()

In [20]:
chain_pars = {'destination_place': PARAMETERS['destination_place'],
              'travel_plan': generated_travel_plan,
              'hotels_json': extracted_hotels,
              'user_preferences':  '',#"the accomodation should be a luxury one and I want to be back in Milan in the afternoon."#'the flights should be with the lowest prices, the accomodation should be with the best price and with good reviews',
             }
print(update_prompt.invoke(chain_pars))

messages=[SystemMessage(content="You are an helpful travel plan assistant that generates travel plans based on user's requestsIt is important that the assistant respects the requested output formats.You must only answer with just the answer to the request, avoiding any kind of conversational sentence."), HumanMessage(content='Below it is provided a proxy (in JSON format) of the travel plan that the user wants to perform:\n{\'Day 1\': {\'Berlin\': [\'Arrive in Berlin (morning)\', \'Check into hotel (midday)\', \'Visit Brandenburg Gate (afternoon)\', \'Dinner at a local restaurant (evening)\']}, \'Day 2\': {\'Berlin\': [\'Visit Berlin Wall Memorial (morning)\', \'Explore Museum Island (midday)\', \'Lunch at a café (afternoon)\', \'Visit Pergamon Museum (afternoon)\', \'Evening stroll along the Spree River (evening)\']}, \'Day 3\': {\'Berlin\': [\'Visit Reichstag Building (morning)\', \'Explore Tiergarten Park (midday)\', \'Lunch in the park (afternoon)\', \'Visit Victory Column (afternoo

In [21]:
hotel_choice_agent = travel_plan_prompt_generator| openai_agent | StrOutputParser() | RunnableLambda(from_json_to_text)

In [22]:
chosen_hotel = hotel_choice_agent.invoke(chain_pars)

print(chosen_hotel)

{'hotel_id': '0', 'reason': 'Best balance of price ($169), high number of reviews (2914), excellent location rating (5.0), and overall rating (4). Close proximity to major attractions like Bode-Museum and Berlin Central Train Station.'}


In [23]:
chosen_hotel_properties = extracted_hotels[int(chosen_hotel['hotel_id'])]
print(chosen_hotel_properties)

{'hotel_id': 0, 'type': 'hotel', 'name': 'MEININGER Hotel Berlin Mitte Humboldthaus', 'description': 'Hip lodging with private rooms & mixed dorms, plus a lounge/snack bar, a guest kitchen & free Wi-Fi.', 'number_reviews': 2914, 'total_cost': 169, 'location_rating': 5.0, 'overall_rating': 4, 'nearby_places': [{'name': 'Bode-Museum', 'transportations': [{'type': 'Walking', 'duration': '7 min'}]}, {'name': 'Berlin Central Train Station', 'transportations': [{'type': 'Taxi', 'duration': '8 min'}]}, {'name': 'Berlin Brandenburg Airport', 'transportations': [{'type': 'Taxi', 'duration': '39 min'}, {'type': 'Public transport', 'duration': '45 min'}]}, {'name': 'Kamala', 'transportations': [{'type': 'Walking', 'duration': '1 min'}]}], 'reviews_breakdown': [{'name': 'Location', 'description': 'Location', 'total_mentioned': 729, 'positive': 644, 'negative': 28, 'neutral': 57}, {'name': 'Sleep', 'description': 'Sleep', 'total_mentioned': 438, 'positive': 129, 'negative': 255, 'neutral': 54}, {'n

In [24]:
chosen_hotel = {"hotel_id": 0, "reason": "Best balance of price ($169), high number of reviews (2914), excellent location rating (5.0), and overall rating (4). Close proximity to major attractions like Bode-Museum and Berlin Central Train Station."}


chosen_hotel_properties = extracted_hotels[int(chosen_hotel['hotel_id'])]
chosen_hotel_properties

{'hotel_id': 0,
 'type': 'hotel',
 'name': 'MEININGER Hotel Berlin Mitte Humboldthaus',
 'description': 'Hip lodging with private rooms & mixed dorms, plus a lounge/snack bar, a guest kitchen & free Wi-Fi.',
 'number_reviews': 2914,
 'total_cost': 169,
 'location_rating': 5.0,
 'overall_rating': 4,
 'nearby_places': [{'name': 'Bode-Museum',
   'transportations': [{'type': 'Walking', 'duration': '7 min'}]},
  {'name': 'Berlin Central Train Station',
   'transportations': [{'type': 'Taxi', 'duration': '8 min'}]},
  {'name': 'Berlin Brandenburg Airport',
   'transportations': [{'type': 'Taxi', 'duration': '39 min'},
    {'type': 'Public transport', 'duration': '45 min'}]},
  {'name': 'Kamala',
   'transportations': [{'type': 'Walking', 'duration': '1 min'}]}],
 'reviews_breakdown': [{'name': 'Location',
   'description': 'Location',
   'total_mentioned': 729,
   'positive': 644,
   'negative': 28,
   'neutral': 57},
  {'name': 'Sleep',
   'description': 'Sleep',
   'total_mentioned': 438,

In [25]:
flights_costs = sum([x['price'] for x in flights])
hotel_cost = chosen_hotel_properties['total_cost']

travel_plan_cost = hotel_cost + flights_costs
travel_plan_cost

364

In [26]:
generated_travel_plan

{'Day 1': {'Berlin': ['Arrive in Berlin (morning)',
   'Check into hotel (midday)',
   'Visit Brandenburg Gate (afternoon)',
   'Dinner at a local restaurant (evening)']},
 'Day 2': {'Berlin': ['Visit Berlin Wall Memorial (morning)',
   'Explore Museum Island (midday)',
   'Lunch at a café (afternoon)',
   'Visit Pergamon Museum (afternoon)',
   'Evening stroll along the Spree River (evening)']},
 'Day 3': {'Berlin': ['Visit Reichstag Building (morning)',
   'Explore Tiergarten Park (midday)',
   'Lunch in the park (afternoon)',
   'Visit Victory Column (afternoon)',
   'Dinner in Kreuzberg (evening)']},
 'Day 4': {'Berlin': ['Visit East Side Gallery (morning)',
   'Explore Friedrichshain district (midday)',
   'Lunch at a local eatery (afternoon)',
   'Visit Jewish Museum (afternoon)',
   'Attend a local event or concert (evening)']},
 'Day 5': {'Berlin': ['Visit Charlottenburg Palace (morning)',
   'Explore the palace gardens (midday)',
   'Lunch at a nearby café (afternoon)',
   'Vi

### Aggregator Agent

In [59]:
#SOFIA VERSION

prompt_generation = """Below it is provided a draft of a travel plan:
{travel_plan}

Below the accomodation chosen for the travel plan
{hotels_json}

Below the transports chosen for the travel plan
{flights_json}

Here are the user preferences for the travel plan (if not provided do not consider the check step): '{user_preferences}'.

TASK 1: add the accomodation and the transport information in the travel plan activities list.
TASK 2: remove (or move in other days if possible the travel activities that can interfer with the time-slots of the transports and the accomodation check-in/check-out times.

Return the travel plan output after TASK 1 and TASK 2.
"""

travel_plan_prompt_generator = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=("You are a great travel plan assistant that satisfies every user's requests"
                          "It is important that the assistant respects the requested output formats."
                          "You must only answer with just the answer to the request, avoiding any kind of conversational sentence.")),
        HumanMessagePromptTemplate.from_template(prompt_generation),
    ]
)

PARAMETERS['gpt_params']['model'] = 'gpt-4o' 

openai_agent = ChatOpenAI(**PARAMETERS['gpt_params'])

update_prompt = travel_plan_prompt_generator| openai_agent | StrOutputParser()

In [60]:
generated_travel_plan = """{'Day 1 (2024-08-30)': {'Berlin': ['Arrive in Berlin (morning)',
   'Check into hotel (midday)',
   'Visit Brandenburg Gate (afternoon)',
   'Dinner at a local restaurant (evening)']},
 'Day 2 (2024-08-31)': {'Berlin': ['Visit Berlin Wall Memorial (morning)',
   'Explore Museum Island (midday)',
   'Lunch at a café (afternoon)',
   'Visit Pergamon Museum (afternoon)',
   'Evening stroll along the Spree River (evening)']},
 'Day 3 (2024-09-01)': {'Berlin': ['Visit Reichstag Building (morning)',
   'Explore Tiergarten Park (midday)',
   'Lunch in the park (afternoon)',
   'Visit Charlottenburg Palace (afternoon)',
   'Dinner in Charlottenburg (evening)']},
 'Day 4 (2024-09-02)': {'Berlin': ['Visit East Side Gallery (morning)',
   'Explore Kreuzberg district (midday)',
   'Lunch at a street food market (afternoon)',
   'Visit Checkpoint Charlie (afternoon)',
   'Evening at a local bar (evening)']},
 'Day 5 (2024-09-03)': {'Berlin': ['Visit Jewish Museum (morning)',
   'Explore Potsdamer Platz (midday)',
   'Lunch at a nearby restaurant (afternoon)',
   'Visit Sony Center (afternoon)',
   'Dinner and a show at a theater (evening)']},
 'Day 6 (2024-09-04)': {'Berlin': ['Day trip to Potsdam (morning)',
   'Visit Sanssouci Palace (midday)',
   'Lunch in Potsdam (afternoon)',
   'Explore the gardens (afternoon)',
   'Return to Berlin (evening)']},
 'Day 7 (2024-09-05)': {'Berlin': ['Visit Berlin Cathedral (morning)',
   'Explore Hackescher Markt (midday)',
   'Lunch at a local café (afternoon)',
   'Last-minute shopping (afternoon)',
   'Depart from Berlin (evening)']}}"""

In [61]:
flights[0]['slot'] = 'morning'
flights[1]['slot'] = 'morning'
flights

[{'flight_id': 0,
  'departure_time': '2024-08-30 06:55',
  'arrival_time': '2024-08-30 08:35',
  'departure_airport': 'Milan Linate Airport',
  'arrival_airport': 'Berlin Brandenburg Airport',
  'airline': 'easyJet',
  'price': 121,
  'total_duration': 100,
  'reason': 'Lowest price and early arrival time in Berlin.',
  'type': 'departure_flight',
  'slot': 'morning'},
 {'flight_id': 3,
  'departure_time': '2024-09-05 08:20',
  'arrival_time': '2024-09-05 10:00',
  'departure_airport': 'Berlin Brandenburg Airport',
  'arrival_airport': 'Milan Linate Airport',
  'airline': 'easyJet',
  'price': 74,
  'total_duration': 100,
  'reason': 'Lowest price and convenient morning departure from Berlin.',
  'type': 'return_flight',
  'slot': 'morning'}]

In [63]:
update_prompt = travel_plan_prompt_generator

chain_pars = {
    'travel_plan': generated_travel_plan,
    'hotels_json': chosen_hotel_properties,
    'flights_json': flights,
    'destination_place': PARAMETERS['destination_place'],
    'n_days': PARAMETERS['n_travel_days'],
    'user_preferences': ''
}

output = update_prompt.invoke(chain_pars)

print(output)

messages=[SystemMessage(content="You are a great travel plan assistant that satisfies every user's requestsIt is important that the assistant respects the requested output formats.You must only answer with just the answer to the request, avoiding any kind of conversational sentence."), HumanMessage(content="Below it is provided a draft of a travel plan:\n{'Day 1 (2024-08-30)': {'Berlin': ['Arrive in Berlin (morning)',\n   'Check into hotel (midday)',\n   'Visit Brandenburg Gate (afternoon)',\n   'Dinner at a local restaurant (evening)']},\n 'Day 2 (2024-08-31)': {'Berlin': ['Visit Berlin Wall Memorial (morning)',\n   'Explore Museum Island (midday)',\n   'Lunch at a café (afternoon)',\n   'Visit Pergamon Museum (afternoon)',\n   'Evening stroll along the Spree River (evening)']},\n 'Day 3 (2024-09-01)': {'Berlin': ['Visit Reichstag Building (morning)',\n   'Explore Tiergarten Park (midday)',\n   'Lunch in the park (afternoon)',\n   'Visit Charlottenburg Palace (afternoon)',\n   'Dinner

In [50]:
chain_pars = {
    'travel_plan': generated_travel_plan,
    'hotels_json': chosen_hotel_properties,
    'flights_json': flights,
    'destination_place': PARAMETERS['destination_place'],
    'n_days': PARAMETERS['n_travel_days'],
    'user_preferences': ''
}

output = update_prompt.invoke(chain_pars)

print(output)

{'Day 1 (2024-08-30)': {'Berlin': ['Flight from Milan Linate Airport to Berlin Brandenburg Airport (06:55 - 08:35)',
   'Arrive in Berlin (morning)',
   'Check into MEININGER Hotel Berlin Mitte Humboldthaus (midday)',
   'Visit Brandenburg Gate (afternoon)',
   'Dinner at a local restaurant (evening)']},
 'Day 2 (2024-08-31)': {'Berlin': ['Visit Berlin Wall Memorial (morning)',
   'Explore Museum Island (midday)',
   'Lunch at a café (afternoon)',
   'Visit Pergamon Museum (afternoon)',
   'Evening stroll along the Spree River (evening)']},
 'Day 3 (2024-09-01)': {'Berlin': ['Visit Reichstag Building (morning)',
   'Explore Tiergarten Park (midday)',
   'Lunch in the park (afternoon)',
   'Visit Charlottenburg Palace (afternoon)',
   'Dinner in Charlottenburg (evening)']},
 'Day 4 (2024-09-02)': {'Berlin': ['Visit East Side Gallery (morning)',
   'Explore Kreuzberg district (midday)',
   'Lunch at a street food market (afternoon)',
   'Visit Checkpoint Charlie (afternoon)',
   'Evening

In [76]:
printgenerated_travel_plan

"{'Day 1': {'Berlin': ['Arrive in Berlin from Milan', 'Check into hotel (30 minutes)', 'Visit Brandenburg Gate (1 hour)', 'Explore the Reichstag Building (1.5 hours)', 'Dinner at a local restaurant (2 hours)']}, 'Day 2': {'Berlin': ['Visit the Berlin Wall Memorial (1.5 hours)', 'Explore Museum Island (3 hours)', 'Lunch at a nearby café (1 hour)', 'Visit the Pergamon Museum (2 hours)', 'Evening stroll in Alexanderplatz (1 hour)']}, 'Day 3': {'Berlin': ['Visit the East Side Gallery (1 hour)', 'Explore Kreuzberg district (2 hours)', 'Lunch at a street food market (1 hour)', 'Visit the Jewish Museum (2 hours)', 'Dinner and nightlife in Friedrichshain (3 hours)']}, 'Day 4': {'Berlin': ['Visit Charlottenburg Palace (2 hours)', 'Explore the Berlin Zoo (2.5 hours)', 'Lunch in the Tiergarten (1 hour)', 'Shopping at Kurfürstendamm (2 hours)', 'Depart from Berlin to Milan']}}"

In [67]:
#SOFIA VERSION

prompt_generation = """Below it is provided a series of activity, divided per day (in JSON format) that represent a possible travel plan for the user:
{travel_plan}
\nIn the JSON below I provide the chosen hotel/s in {destination_place}
{hotels_json}
\n In the JSON below the transports chosen by the travel assistant (you)
{flights_json}
\nHere are the user preferences for the travel plan (if not provided do not consider the check step): '{user_preferences}'.
Please check that the selected accomodation and flights satisfies the user preferences and then incorporate those information in the travel plan.
If necessary update (either redistributing the activities or eliminating less important ones) the travel plan in order to respect the trasports time of arrivals and departurte times and allow enought time to do the activity.
Return the updated travel plan.
"""

travel_plan_prompt_generator = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=("You are an helpful travel plan assistant that generates travel plans based on user's requests"
                          "It is important that the assistant respects the requested output formats."
                          "You must only answer with just the answer to the request, avoiding any kind of conversational sentence.")),
        HumanMessagePromptTemplate.from_template(prompt_generation),
    ]
)

PARAMETERS['gpt_params']['model'] = 'gpt-4o-mini' 

openai_agent = ChatOpenAI(**PARAMETERS['gpt_params'])

update_prompt = travel_plan_prompt_generator| openai_agent | StrOutputParser()

{'Day 1': {'Berlin': ['Arrive in Berlin from Milan (Flight: easyJet, 06:55 - 08:35)', 'Check into MEININGER Hotel Berlin Mitte Humboldthaus (30 minutes)', 'Visit Brandenburg Gate (1 hour)', 'Explore the Reichstag Building (1.5 hours)', 'Dinner at a local restaurant (2 hours)']}, 'Day 2': {'Berlin': ['Visit the Berlin Wall Memorial (1.5 hours)', 'Explore Museum Island (3 hours)', 'Lunch at a nearby café (1 hour)', 'Visit the Pergamon Museum (2 hours)', 'Evening stroll in Alexanderplatz (1 hour)']}, 'Day 3': {'Berlin': ['Visit the East Side Gallery (1 hour)', 'Explore Kreuzberg district (2 hours)', 'Lunch at a street food market (1 hour)', 'Visit the Jewish Museum (2 hours)', 'Dinner and nightlife in Friedrichshain (3 hours)']}, 'Day 4': {'Berlin': ['Visit Charlottenburg Palace (2 hours)', 'Explore the Berlin Zoo (2.5 hours)', 'Lunch in the Tiergarten (1 hour)', 'Shopping at Kurfürstendamm (2 hours)', 'Check out from MEININGER Hotel Berlin Mitte Humboldthaus (30 minutes)', 'Depart from B

In [73]:
from_json_to_text(output)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)